# Introduction

**Welcome to the project!** Here, I am going to present my work on the Home Credit dataset. This little message serves as an introduction to how everything here works.

The overall task is to analyze the dataset, which can be found under this link, and is noticeably absent from the repository. This is because the data file is absolutely massive and would exceed the free version of GitHub version. The implementation here is very straightforward and you should understand the logic behind what is what pretty easily, however:
 
**Markdown**

As the requirement to use the Markdown as if I am talking to a manager is now gone, I was using it pretty casually. They are still pretty useful and are on point, so the casual style would not diminish their usefulness.

**Code**

I tried to make the _Code_ sections of the work as readable and intuitive and possible, so if some piece of code is repeated several times. a *support_file* library is used, and the reader is adviced to take a look at it if something does not make sense.

**Visualizations**

The project contains visualizations in every part subjectively deemed important. For this, matplotlib and seaborn libraries are used.

# Initial Dataset Overview

In [ ]:
import support_file as sf
import importlib
from sklearn.metrics import accuracy_score
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_curve, precision_recall_curve
from sklearn.metrics import fbeta_score
from sklearn.metrics import roc_auc_score
import pandas as pd, matplotlib.pyplot as plt, seaborn as sns, numpy as np
from scipy import stats
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.metrics import f1_score
from statsmodels.stats.proportion import proportions_ztest
from sklearn.model_selection import cross_val_score, StratifiedKFold
from pandas.api.types import CategoricalDtype
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
import gender_guesser.detector as gender
import pandas as pd
import optuna
import warnings
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import OrdinalEncoder
from category_encoders import TargetEncoder
from scipy.stats import ks_2samp
from sklearn.neighbors import KNeighborsRegressor
import numpy as np
from scipy.stats import mannwhitneyu
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from scipy.stats import normaltest
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
from phik import phik_matrix
from phik.report import plot_correlation_matrix
from xgboost import XGBClassifier
import shap
from sklearn.metrics import average_precision_score
import gc
import duckdb
from sklearn.preprocessing import StandardScaler
import umap
import hdbscan
from sklearn.impute import SimpleImputer
import tensorflow as tf
import h5py
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import HistGradientBoostingRegressor
import gower
import umap
import hdbscan
import numpy as np
import pandas as pd
import lightgbm as lgb
import umap
import hdbscan
from sklearn.preprocessing import OrdinalEncoder
import scipy.sparse as sp
from sklearn.feature_extraction import FeatureHasher

The very first problem that we should solve is to understand the data that we are dealing with. Here we have, well, more than one dataframe to work with, so it is better to start with some exploration of what we are actually working with before agglomerating it into one big dataset

## Application Data

In [ ]:
importlib.reload(sf)
np.random.seed(42)
application_train = pd.read_csv('application_train.csv')
application_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


This dataset contains information on **loan applications** submitted by clients to Home Credit. Each row represents a **single loan application** made by a client, including details about the applicant’s **demographics**, **financial situation**, and **creditworthiness** at the time of application. It also includes a binary `TARGET` column indicating whether the applicant later **experienced payment difficulties**.

This is the **central table** in the dataset and serves as the **main anchor** for merging with other files. It’s uniquely identified by `SK_ID_CURR` and links to behavioral and historical records through various keys.

Included Data Categories:

- **Demographic information** (e.g., age, gender, family status, housing type)
- **Financial characteristics** (e.g., income, credit amount, annuity)
- **Asset ownership** (e.g., car, real estate)
- **Employment and registration history**
- **Building characteristics** (normalized features from housing registry)
- **Contact and document submission flags**
- **External risk scores** (from external providers)
- **Social circle statistics**
- **Credit Bureau inquiry counts**

Connections:
- Can be joined with:
  - `bureau.csv` (via `SK_ID_CURR`)
  - `previous_application.csv` (via `SK_ID_CURR`)
  - All other behavioral tables through linked previous loans

## Bureau Balance

In [ ]:
bureau_balance = pd.read_csv('bureau_balance.csv')
bureau_balance.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


This dataset contains **monthly snapshots** of the status of credit lines that were reported to the **Credit Bureau** and linked to each client in the main application sample. Each row represents **one month of history for a single credit record**, tracking how the credit’s status evolved over time.

This file provides **behavioral data** on credit usage and repayment performance prior to the loan application date. It helps model **credit behavior over time**, such as delinquencies or closures.

It is a **child table** of `bureau.csv`, linked via `SK_ID_BUREAU`.

Included Data Categories:

- Monthly time step relative to the current application date
- Credit status per month (e.g., active, delinquent, closed)

Connections:
- Linked to `bureau.csv` via `SK_ID_BUREAU` (many-to-one relationship)

## Bureau Data

In [ ]:
bureau = pd.read_csv('bureau.csv')
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


This dataset contains information on the **client’s previous and current credit obligations** with **external financial institutions**, as reported to a Credit Bureau. Each row represents a **single credit line** (e.g. consumer loan, credit card, microloan) held by a client prior to or at the time of the Home Credit loan application.

It provides a snapshot of the client’s **repayment history**, **credit utilization**, **credit type**, and **status** (active or closed) for each of these third-party credits.

This is a **secondary table**, connected to the main application data through `SK_ID_CURR`.

Included Data Categories:

- Status and timeline of each credit line (e.g., active, closed, overdue)
- Credit amounts and remaining debt
- Dates of credit issuance and closure
- Type of credit product used
- Past delinquencies, credit limits, and prolongations

Connections:
- Linked to `application_train.csv` via `SK_ID_CURR` (many-to-one)
- Linked to `bureau_balance.csv` via `SK_ID_BUREAU` (one-to-many)

## Credit Card Balance

In [ ]:
credit_card_balance = pd.read_csv('credit_card_balance.csv')
credit_card_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


This dataset contains **monthly balance snapshots** of clients’ **credit card accounts** held with Home Credit. Each row corresponds to a **specific month of activity** for one credit card contract, providing a time-series view of card usage, repayments, limits, and delinquencies.

This is a **behavioral table** that helps assess a client's recent **credit card behavior** leading up to their loan application. It tracks both **financial activity** and **repayment discipline** over time.

Included Data Categories:

- Credit card balances and credit limits
- Amounts drawn (ATM, POS, others)
- Minimum due payments and actual payments made
- Number of transactions per month
- Days past due and contract status (e.g., Active, Completed)

Connections:
- Linked to `previous_application.csv` via `SK_ID_PREV` (many-to-one)
- Linked to `application_train.csv` via `SK_ID_CURR` (many-to-one)

## Installments Payments

In [ ]:
installments_payments = pd.read_csv('installments_payments.csv')
installments_payments.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


This dataset contains the **repayment history of previous loans** taken by clients. Each row corresponds to a **single installment** of a previous credit, capturing both the **scheduled payment date and amount**, as well as the **actual date and amount paid**.

It offers a granular view of a client’s **repayment behavior**, making it valuable for analyzing **payment punctuality**, **delinquencies**, or **prepayments**.

This is a **time-series table** at the **transaction level**, connected through both the client and the specific previous credit contract.

Included Data Categories:

- Scheduled vs actual payment timing (relative to current application date)
- Expected installment vs actual payment
- Installment versioning and sequence of payments

Connections:
- Linked to `previous_application.csv` via `SK_ID_PREV` (many-to-one)
- Linked to `application_train.csv` via `SK_ID_CURR` (many-to-one)

## Point-of-Sale Balance

In [ ]:
POS_CASH_balance = pd.read_csv('POS_CASH_balance.csv')
POS_CASH_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


This dataset contains **monthly snapshots of POS (point-of-sale) and cash loans** that clients previously held with Home Credit. Each row corresponds to a **monthly record** for a single POS or cash loan, tracking the loan’s **status, installment schedule**, and **repayment behavior** over time.

It functions as a **time-series table** capturing repayment progress and delinquencies on previous Home Credit products, and is useful for assessing a client’s historical **loan performance**.

Included Data Categories:

- Number of total and remaining installments
- Contract status (active, completed, etc.)
- Monthly delinquency flags (with and without tolerance)
- Relative time of snapshot to loan application

Connections:
- Linked to `previous_application.csv` via `SK_ID_PREV` (many-to-one)
- Linked to `application_train.csv` via `SK_ID_CURR` (many-to-one)

## Previous Loan Application Data

In [ ]:
previous_application = pd.read_csv('previous_application.csv')
previous_application.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


This dataset contains data on all **previous loan applications** submitted by clients to Home Credit before the current loan recorded in `application_train.csv`. Each row represents a **single past application**, whether it was approved, rejected, or canceled, and includes information about the **requested amount**, **final approved credit**, **purpose**, **timing**, and **loan product type**.

This table provides insight into a client’s **past borrowing behavior**, including whether they were a repeat applicant, what kind of products they sought, and how their previous loans were structured.

Included Data Categories:

- Type and purpose of previously requested loans
- Amounts applied for and granted
- Timing of application and contractual terms
- Status of the application (approved, refused, etc.)
- Product/channel metadata (e.g., seller, yield group, contract combination)
- Scheduled disbursement and due dates (relative to current loan)

Connections:
- Linked to `application_train.csv` via `SK_ID_CURR` (many-to-one)
- Key for linking to `installments_payments.csv`, `POS_CASH_balance.csv`, and `credit_card_balance.csv` via `SK_ID_PREV`

# Approach

In this part, we are going to optimize the work ahead, which would involve dropping redundant features and agglomeration, and, considering the data we are dealing with, I want to make sure that there is no data leakage, which is why the data split is going to happen in the very beginning, and all the decisions on whether or not to keep a feature are going to be done exclusively on the training set. To simplify my life, I am not going to be splitting dataframes other than `application_train` and `application_test` if there is not training. For example, if I am changing datatypes or mapping, as I am basing it on metadata, which is shared between the two, I am not going to do it twice

In [ ]:
X = application_train.drop(columns=['TARGET'])
y = application_train['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

# Balance Branch

## Bureau Balance

In [ ]:
bureau_balance.sort_values(by=['SK_ID_BUREAU', 'MONTHS_BALANCE'], inplace=True)
bureau_balance.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
26079849,5001709,-96,X
26079848,5001709,-95,X
26079847,5001709,-94,X
26079846,5001709,-93,X
26079845,5001709,-92,X


In [ ]:
bureau_balance.dtypes

SK_ID_BUREAU       int64
MONTHS_BALANCE     int64
STATUS            object
dtype: object

In [ ]:
bureau_balance['STATUS'].value_counts()

STATUS
C    13646993
0     7499507
X     5810482
1      242347
5       62406
2       23419
3        8924
4        5847
Name: count, dtype: int64

In [ ]:
importlib.reload(sf)

filtered_balance = bureau_balance.copy()
filtered_balance = filtered_balance[~((filtered_balance['STATUS'] == 'X') & (filtered_balance['MONTHS_BALANCE'] <= -24))]

status_pivot = (
    filtered_balance
    .groupby(['SK_ID_BUREAU', 'STATUS'])
    .size()
    .unstack(fill_value=0)
)

for col in ['0', '1', '2', '3', '4', '5', 'C', 'X']:
    if col not in status_pivot.columns:
        status_pivot[col] = 0

status_pivot['n'] = status_pivot[['0', '1', '2', '3', '4', '5']].sum(axis=1)
status_pivot['n_total'] = status_pivot[['0', '1', '2', '3', '4', '5', 'X']].sum(axis=1)
status_pivot['pct_1'] = status_pivot['1'] / status_pivot['n']
status_pivot['pct_12'] = (status_pivot['1'] + status_pivot['2']) / status_pivot['n']
status_pivot['pct_x'] = status_pivot['X'] / status_pivot['n_total']

status_pivot['BUREAU_BALANCE_CLASS'] = status_pivot.apply(sf.classify_bureau_balance, axis=1)
result = status_pivot[['BUREAU_BALANCE_CLASS']].reset_index()

In [ ]:
result["BUREAU_BALANCE_CLASS"].value_counts()

BUREAU_BALANCE_CLASS
perfect    570182
unknown    138327
good        50457
risky       39947
okay        12426
Name: count, dtype: int64

I should mention a small note on how this classification works, as I spent far too much take making it sensible. If 80% or more of the statuses, after excluding entries older than 24 months, are 'X', we label it unknown. If all statuses are '0', it's 'perfect'. If the loan only includes '0' and '1', and '1' makes up less than 15%, it is 'good'. If the statuses include only '0', '1', and '2', and '1' + '2' together are under 20%, 'okay'. Anything else results in 'risky'

Now that we finally have this table prepared, we will attach it to the transformed `bureau` table, as, for now, there is little we can do about it

Here we can see that we have a lot of people with the status unknown. I would assume that there is really very little that we can do about it. Since, and I am referring to the data source here, this data is from a third-party source, the 'bureau', whatever it means, there is no way we could impute the 'unknown' status here.

## Bureau

In [ ]:
bureau = bureau.merge(result, on='SK_ID_BUREAU', how='left')
bureau.drop(columns=['SK_ID_BUREAU'], inplace=True)

In [ ]:
del result, status_pivot, filtered_balance, X, y
gc.collect()

20

In [ ]:
bureau['BUREAU_BALANCE_CLASS'].isna().sum()/bureau.shape[0]

np.float64(0.5516386355850639)

Seems like not everyone has had past history with the bureau

In [ ]:
bureau.head()

,SK_ID_CURR,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,BUREAU_BALANCE_CLASS
0,215354,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,NaN
1,215354,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,NaN
2,215354,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,NaN
3,215354,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN,NaN
4,215354,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,NaN


Now what we are going to do is that we are going to split our data into categorical and continuous features to try to analyze what features are actually important. This is done first and foremost to save memory, as the entire dataset is really heavy and there had been some issues with trying to run it on my laptop

Now I should explain the approach I would like to take with this. Later on, I will be using not the most intuitive methods to agglomerate the tables by `SK_ID_CURR` but, after very carefully inspecting the features, it seems like there would not be any need in this, as it is pretty straightforward. With that, the plan here is to invent some features, test their effectiveness, and leave the best ones

In [ ]:
bureau['CREDIT_DAY_OVERDUE'].max()

np.int64(2792)

### Feature Engineering

In [ ]:
bureau['CREDIT_TYPE'] = (
    bureau['CREDIT_TYPE']
    .replace({'Unknown type of loan': np.nan})
    .astype('category')
)

bureau['BUREAU_BALANCE_CLASS'] = bureau['BUREAU_BALANCE_CLASS'].astype('category')

In [ ]:
warnings.filterwarnings("ignore")
np.seterr(all='ignore')

def most_frequent_with_tiebreaker(series, global_freq):
    top = series.value_counts().nlargest(2)
    if len(top) == 1:
        return top.index[0]
    top_candidates = [val for val in top.index if val in global_freq.index]
    if len(top_candidates) == 0:
        return np.nan
    return global_freq.loc[top_candidates].idxmax()

currency_global_freq = bureau['CREDIT_CURRENCY'].value_counts()
type_global_freq = bureau['CREDIT_TYPE'].value_counts()

bureau['CREDIT_ACTIVE'] = bureau['CREDIT_ACTIVE'].fillna("Unknown")
bureau['CREDIT_CURRENCY'] = bureau['CREDIT_CURRENCY'].fillna("Unknown")
if pd.api.types.is_categorical_dtype(bureau['CREDIT_TYPE']):
    bureau['CREDIT_TYPE'] = bureau['CREDIT_TYPE'].cat.add_categories(['Unknown'])

bureau['CREDIT_TYPE'] = bureau['CREDIT_TYPE'].fillna('Unknown')


grouped = bureau.groupby('SK_ID_CURR')

bureau_agg = pd.DataFrame(index=grouped.size().index)

bureau_agg['NUM_ACTIVE_CREDIT'] = grouped['CREDIT_ACTIVE'].apply(lambda x: (x == 'Active').sum())

bureau_agg['CREDIT_CURRENCY_MODE'] = grouped['CREDIT_CURRENCY'].apply(lambda x: most_frequent_with_tiebreaker(x, currency_global_freq))

bureau_agg['DAYS_CREDIT_MIN'] = grouped['DAYS_CREDIT'].max()
bureau_agg['DAYS_CREDIT_MAX'] = grouped['DAYS_CREDIT'].min()
bureau_agg['DAYS_CREDIT_MEDIAN'] = grouped['DAYS_CREDIT'].median()

bureau_agg['NUMBER_OVERDUE'] = grouped['CREDIT_DAY_OVERDUE'].apply(lambda x: (x.fillna(0) > 0).sum())
bureau_agg['OVERDUE_MAX'] = grouped['CREDIT_DAY_OVERDUE'].max()

bureau_agg['DAYS_ENDDATE_MAX'] = grouped['DAYS_CREDIT_ENDDATE'].max()

bureau_agg['ENDDATE_DIFF_COUNT'] = grouped.apply(lambda x: (x['DAYS_CREDIT_ENDDATE'] != x['DAYS_ENDDATE_FACT']).sum())
bureau_agg['ENDDATE_DIFF_MEAN'] = grouped.apply(lambda x: (x['DAYS_CREDIT_ENDDATE'] - x['DAYS_ENDDATE_FACT']).mean())
bureau_agg['ENDDATE_DIFF_MAX'] = grouped.apply(lambda x: (x['DAYS_CREDIT_ENDDATE'] - x['DAYS_ENDDATE_FACT']).max())

bureau_agg['AMT_CREDIT_MAX_OVERDUE_MEAN'] = grouped['AMT_CREDIT_MAX_OVERDUE'].mean()
bureau_agg['AMT_CREDIT_MAX_OVERDUE_MAX'] = grouped['AMT_CREDIT_MAX_OVERDUE'].max()

bureau_agg['CNT_CREDIT_PROLONG_SUM'] = grouped['CNT_CREDIT_PROLONG'].sum()
bureau_agg['CNT_CREDIT_PROLONG_MEAN'] = grouped['CNT_CREDIT_PROLONG'].mean()
bureau_agg['CNT_CREDIT_PROLONG_NONZERO'] = grouped['CNT_CREDIT_PROLONG'].apply(lambda x: (x != 0).sum())

def compute_amt_credit_sum_ratio(df):
    valid = df['DAYS_CREDIT_ENDDATE'].notna()
    denom = df['DAYS_CREDIT'] + df['DAYS_CREDIT_ENDDATE']
    ratio = np.where(valid, np.abs(df['DAYS_CREDIT_ENDDATE']) / denom.replace(0, np.nan), 0)
    ratio = np.nan_to_num(ratio)
    return ratio.sum()

bureau_agg['CREDIT_SUM_RATIO'] = grouped.apply(compute_amt_credit_sum_ratio)

bureau_agg['AMT_CREDIT_SUM_LIMIT_MAX'] = grouped['AMT_CREDIT_SUM_LIMIT'].max()

bureau_agg['CREDIT_TYPE_MODE'] = grouped['CREDIT_TYPE'].apply(lambda x: most_frequent_with_tiebreaker(x, type_global_freq))

bureau_agg['AMT_ANNUITY_POS_ENDDATE_SUM'] = grouped.apply(
    lambda x: x.loc[x['DAYS_CREDIT_ENDDATE'] > 0, 'AMT_ANNUITY'].sum()
)

bureau_agg['BUREAU_BALANCE_CLASS'] = grouped['BUREAU_BALANCE_CLASS'].first()

bureau_agg['NUM_CLOSED_CREDIT'] = grouped['CREDIT_ACTIVE'].apply(lambda x: (x == 'Closed').sum())

def custom_median_enddate(series):
    non_na_series = series.dropna()
    if non_na_series.empty:
        return np.nan
    if (non_na_series > 0).all():
        return non_na_series.median()
    elif (non_na_series < 0).all():
        return 0
    else:
        return non_na_series.median()

bureau_agg['DAYS_ENDDATE_MEDIAN_CUSTOM'] = grouped['DAYS_CREDIT_ENDDATE'].apply(custom_median_enddate)

def custom_mean_enddate(series):
    non_na_series = series.dropna()
    if non_na_series.empty:
        return np.nan
    if (non_na_series > 0).all():
        return non_na_series.mean()
    elif (non_na_series < 0).all():
        return 0
    else:
        return non_na_series.mean()

bureau_agg['DAYS_ENDDATE_MEAN_CUSTOM'] = grouped['DAYS_CREDIT_ENDDATE'].apply(custom_mean_enddate)

bureau_agg['DAYS_ENDDATE_MIN'] = grouped['DAYS_CREDIT_ENDDATE'].min()

bureau_agg['DAYS_ENDDATE_FACT_CREDIT_DIFF_MEAN'] = grouped.apply(lambda x: (x['DAYS_ENDDATE_FACT'] - x['DAYS_CREDIT']).mean())

bureau_agg['DAYS_ENDDATE_FACT_CREDIT_DIFF_MEDIAN'] = grouped.apply(lambda x: (x['DAYS_ENDDATE_FACT'] - x['DAYS_CREDIT']).median())

bureau_agg['DAYS_ENDDATE_FACT_CREDIT_DIFF_MAX'] = grouped.apply(lambda x: (x['DAYS_ENDDATE_FACT'] - x['DAYS_CREDIT']).max())


def calculate_early_closure_percentage(df_group):
    comparable_rows = df_group.dropna(subset=['DAYS_ENDDATE_FACT', 'DAYS_CREDIT_ENDDATE'])
    if comparable_rows.empty:
        return np.nan
    
    early_closure_count = (comparable_rows['DAYS_ENDDATE_FACT'] > comparable_rows['DAYS_CREDIT_ENDDATE']).sum()
    
    total_comparable_count = len(comparable_rows)
    
    if total_comparable_count == 0:
        return np.nan
    return early_closure_count / total_comparable_count

bureau_agg['PERCENTAGE_EARLY_CLOSURE'] = grouped.apply(calculate_early_closure_percentage)


def calculate_late_closure_percentage(df_group):
    comparable_rows = df_group.dropna(subset=['DAYS_ENDDATE_FACT', 'DAYS_CREDIT_ENDDATE'])
    if comparable_rows.empty:
        return np.nan
    
    late_closure_count = (comparable_rows['DAYS_ENDDATE_FACT'] < comparable_rows['DAYS_CREDIT_ENDDATE']).sum()
    
    total_comparable_count = len(comparable_rows)

    if total_comparable_count == 0:
        return np.nan
    return late_closure_count / total_comparable_count

bureau_agg['PERCENTAGE_LATE_CLOSURE'] = grouped.apply(calculate_late_closure_percentage)

bureau_agg['AMT_CREDIT_MAX_OVERDUE_MEDIAN'] = grouped['AMT_CREDIT_MAX_OVERDUE'].median()

def calculate_ratio_series(numerator_series, denominator_series):
    safe_denominator = denominator_series.replace(0, np.nan)
    ratio = numerator_series / safe_denominator
    return ratio

bureau_agg['DEBT_TO_CREDIT_SUM_MEAN'] = grouped.apply(
    lambda x: calculate_ratio_series(x['AMT_CREDIT_SUM_DEBT'], x['AMT_CREDIT_SUM']).mean()
)

bureau_agg['OVERDUE_TO_CREDIT_SUM_MEAN'] = grouped.apply(
    lambda x: calculate_ratio_series(x['AMT_CREDIT_SUM_OVERDUE'], x['AMT_CREDIT_SUM']).mean()
)

bureau_agg['OVERDUE_TO_DEBT_SUM_MEAN'] = grouped.apply(
    lambda x: calculate_ratio_series(x['AMT_CREDIT_SUM_OVERDUE'], x['AMT_CREDIT_SUM_DEBT']).mean()
)

bureau_agg['CREDIT_UTILIZATION_MEAN'] = grouped.apply(
    lambda x: calculate_ratio_series(x['AMT_CREDIT_SUM_DEBT'], x['AMT_CREDIT_SUM_LIMIT']).mean()
)

bureau_agg['CREDIT_UTILIZATION_MEDIAN'] = grouped.apply(
    lambda x: calculate_ratio_series(x['AMT_CREDIT_SUM_DEBT'], x['AMT_CREDIT_SUM_LIMIT']).median()
)

bureau_agg['DEBT_TO_CREDIT_SUM_MEDIAN'] = grouped.apply(
    lambda x: calculate_ratio_series(x['AMT_CREDIT_SUM_DEBT'], x['AMT_CREDIT_SUM']).median()
)

bureau_agg['OVERDUE_TO_CREDIT_SUM_MEDIAN'] = grouped.apply(
    lambda x: calculate_ratio_series(x['AMT_CREDIT_SUM_OVERDUE'], x['AMT_CREDIT_SUM']).median()
)

bureau_agg['OVERDUE_TO_DEBT_SUM_MEDIAN'] = grouped.apply(
    lambda x: calculate_ratio_series(x['AMT_CREDIT_SUM_OVERDUE'], x['AMT_CREDIT_SUM_DEBT']).median()
)

bureau_agg['DEBT_TO_CREDIT_SUM_MAX'] = grouped.apply(
    lambda x: calculate_ratio_series(x['AMT_CREDIT_SUM_DEBT'], x['AMT_CREDIT_SUM']).max()
)

bureau_agg['OVERDUE_TO_CREDIT_SUM_MAX'] = grouped.apply(
    lambda x: calculate_ratio_series(x['AMT_CREDIT_SUM_OVERDUE'], x['AMT_CREDIT_SUM']).max()
)

bureau_agg['OVERDUE_TO_DEBT_SUM_MAX'] = grouped.apply(
    lambda x: calculate_ratio_series(x['AMT_CREDIT_SUM_OVERDUE'], x['AMT_CREDIT_SUM_DEBT']).max()
)

bureau_agg['CNT_CREDIT_PROLONG_MEDIAN'] = grouped['CNT_CREDIT_PROLONG'].median()

bureau_agg['CNT_CREDIT_PROLONG_MAX'] = grouped['CNT_CREDIT_PROLONG'].max()

bureau_agg['DAYS_ENDDATE_DIFF_STD'] = grouped.apply(
    lambda x: (x['DAYS_CREDIT_ENDDATE'] - x['DAYS_ENDDATE_FACT']).std()
)

def compute_weighted_amt_credit_sum_ratio(df_group):
    valid = df_group['DAYS_CREDIT_ENDDATE'].notna()
    denom = (df_group['DAYS_CREDIT'] + df_group['DAYS_CREDIT_ENDDATE']).replace(0, np.nan)
    ratio = np.where(valid, np.abs(df_group['DAYS_CREDIT_ENDDATE']) / denom, 0)
    ratio = np.nan_to_num(ratio)

    weighted_sum = (ratio * df_group['AMT_CREDIT_SUM'].fillna(0)).sum()
    return weighted_sum

bureau_agg['CREDIT_SUM_RATIO_WEIGHTED'] = grouped.apply(compute_weighted_amt_credit_sum_ratio)

In [ ]:
bureau_agg.head()

,SK_ID_CURR,NUM_ACTIVE_CREDIT,CREDIT_CURRENCY_MODE,DAYS_CREDIT_MIN,DAYS_CREDIT_MAX,DAYS_CREDIT_MEDIAN,NUMBER_OVERDUE,OVERDUE_MAX,DAYS_ENDDATE_MAX,ENDDATE_DIFF_COUNT,...,DEBT_TO_CREDIT_SUM_MEDIAN,OVERDUE_TO_CREDIT_SUM_MEDIAN,OVERDUE_TO_DEBT_SUM_MEDIAN,DEBT_TO_CREDIT_SUM_MAX,OVERDUE_TO_CREDIT_SUM_MAX,OVERDUE_TO_DEBT_SUM_MAX,CNT_CREDIT_PROLONG_MEDIAN,CNT_CREDIT_PROLONG_MAX,DAYS_ENDDATE_DIFF_STD,CREDIT_SUM_RATIO_WEIGHTED
0,100001,3,currency 1,-49,-1572,-857.0,0,0,1778.0,7,...,0.000000,0.0,0.0,0.987405,0.0,0.0,0.0,0,334.957709,1.877024e+06
1,100002,2,currency 1,-103,-1437,-1042.5,0,0,780.0,6,...,0.000000,0.0,0.0,0.546180,0.0,0.0,0.0,0,437.954678,-1.448078e+06
2,100003,1,currency 1,-606,-2586,-1205.5,0,0,1216.0,3,...,0.000000,0.0,NaN,0.000000,0.0,NaN,0.0,0,253.714406,1.550448e+06
3,100004,0,currency 1,-408,-1326,-867.0,0,0,-382.0,1,...,0.000000,0.0,NaN,0.000000,0.0,NaN,0.0,0,62.225397,-7.498313e+04
4,100005,2,currency 1,-62,-373,-137.0,0,0,1324.0,3,...,0.848974,0.0,0.0,0.954794,0.0,0.0,0.0,0,NaN,6.801493e+05


There is a really big chunk of code above but it os not that complicated actually. The only thing that I should clear up is that whenever we are searching for the most frequent value, in case of ties, the most frequent value in the dataframe itself is used. Overall, looking at the feature names would provide a good understanding of what the feature represents

In [ ]:
bureau_agg = pd.read_parquet('bureau_agg.parquet')

In [ ]:
bureau_agg = bureau_agg.reset_index()

bureau_train = bureau_agg[
    bureau_agg['SK_ID_CURR'].isin(set(y_train.index))
].copy()

bureau_test = bureau_agg[
    bureau_agg['SK_ID_CURR'].isin(set(y_test.index))
].copy()

bureau_train['TARGET'] = bureau_train['SK_ID_CURR'].map(y_train)
bureau_test['TARGET'] = bureau_test['SK_ID_CURR'].map(y_test)

In [ ]:
categorical_cols = bureau_train.select_dtypes(include=["category", "object", "bool"]).columns.tolist()
categorical_cols = [col for col in categorical_cols if col not in ('TARGET', 'SK_ID_PREV', 'SK_ID_CURR')]
continuous_cols = bureau_train.select_dtypes(include=["float", "int"]).columns.tolist()
continuous_cols = [col for col in continuous_cols if col not in ('TARGET', 'SK_ID_PREV', 'SK_ID_CURR')]

### Testing!

#### Mutual Information

In [ ]:
sf.compare_mi(bureau_train[continuous_cols + ['TARGET']], 'TARGET')

/home/ilyac/Desktop/Programming/Credit/support_file.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([results_df, new_result], ignore_index=True)
/home/ilyac/Desktop/Programming/Credit/.venv/lib/python3.10/site-packages/sklearn/metrics/cluster/_supervised.py:49: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  type_label = type_of_target(labels_true)
/home/ilyac/Desktop/Programming/Credit/.venv/lib/python3.10/site-packages/sklearn/metrics/cluster/_supervised.py:49: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  type_label = type_of_target(labels_true)
/home/ilyac/Desktop/Programming/Credit/.venv/lib/python3.10/site-packages/sklearn/metri

,Feature,Score
0,CREDIT_SUM_RATIO_WEIGHTED,0.277369
1,CREDIT_SUM_RATIO,0.274431
2,DEBT_TO_CREDIT_SUM_MEAN,0.194568
3,DEBT_TO_CREDIT_SUM_MAX,0.172262
4,DAYS_ENDDATE_MEAN_CUSTOM,0.131336
5,DAYS_ENDDATE_DIFF_STD,0.124561
6,DEBT_TO_CREDIT_SUM_MEDIAN,0.090340
7,ENDDATE_DIFF_MEAN,0.066932
8,AMT_CREDIT_MAX_OVERDUE_MEAN,0.064465
9,DAYS_ENDDATE_FACT_CREDIT_DIFF_MEAN,0.055646


Here we can see that we have a lot of significant features, meaning that we did a good job with engineering

In [ ]:
sf.compare_mi(bureau_train[categorical_cols + ['TARGET']], 'TARGET')

/home/ilyac/Desktop/Programming/Credit/support_file.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([results_df, new_result], ignore_index=True)


,Feature,Score
0,CREDIT_TYPE_MODE,0.000017
1,BUREAU_BALANCE_CLASS,0.000014
2,CREDIT_CURRENCY_MODE,0.000001


The scores are really small here

#### XGBoost Feature Importance

In [ ]:
sf.compare_xgb(bureau_train[continuous_cols + ['TARGET']], 'TARGET')

/home/ilyac/Desktop/Programming/Credit/support_file.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([results_df, new_result], ignore_index=True)


,Feature,Score
0,CREDIT_SUM_RATIO,0.063194
1,DAYS_ENDDATE_DIFF_STD,0.055945
2,AMT_ANNUITY_POS_ENDDATE_SUM,0.051658
3,DAYS_ENDDATE_MIN,0.047705
4,DAYS_ENDDATE_FACT_CREDIT_DIFF_MAX,0.044739
5,DAYS_CREDIT_MEDIAN,0.040695
6,CREDIT_SUM_RATIO_WEIGHTED,0.040074
7,DAYS_ENDDATE_FACT_CREDIT_DIFF_MEAN,0.039634
8,CREDIT_UTILIZATION_MEAN,0.037549
9,DEBT_TO_CREDIT_SUM_MAX,0.036881


Some of them are actually decent here

#### SHAP

In [ ]:
sf.compare_shap(bureau_train[continuous_cols + ['TARGET']], 'TARGET')

 98%|===================| 157896/160391 [00:52<00:00]        

,Feature,Score
0,DAYS_ENDDATE_MEDIAN_CUSTOM,0.065834
1,DAYS_ENDDATE_MIN,0.065626
2,DAYS_ENDDATE_FACT_CREDIT_DIFF_MAX,0.036412
3,DAYS_CREDIT_MAX,0.034261
4,DAYS_ENDDATE_FACT_CREDIT_DIFF_MEDIAN,0.031694
5,DAYS_ENDDATE_FACT_CREDIT_DIFF_MEAN,0.030192
6,DAYS_ENDDATE_MAX,0.014621
7,ENDDATE_DIFF_MEAN,0.010469
8,CREDIT_SUM_RATIO,0.008768
9,AMT_ANNUITY_POS_ENDDATE_SUM,0.008492


Values at the very top are extremely good for this dataset and SHAP

#### Summary

Here we want to trim down our data but not too much, as there is some hope that we are going to find decent interaction terms once we combine all the data, so we dont want to be too harsh. What we are going to do is we are going to keep the features that were near the top of the list in at least one of the test and have shown reasonably high values, which are pretty lax criteria

In [ ]:
to_keep = [
    'SK_ID_CURR',
    'DAYS_ENDDATE_FACT_CREDIT_DIFF_MAX',
    'DAYS_ENDDATE_FACT_CREDIT_DIFF_MEAN',
    'DAYS_ENDDATE_MAX',
    'CREDIT_UTILIZATION_MEAN',
    'CREDIT_UTILIZATION_MEDIAN',
    'DAYS_ENDDATE_FACT_CREDIT_DIFF_MEDIAN',
    'AMT_CREDIT_MAX_OVERDUE_MEDIAN',
    'DEBT_TO_CREDIT_SUM_MEDIAN',
    'DEBT_TO_CREDIT_SUM_MEAN',
    'AMT_CREDIT_MAX_OVERDUE_MEAN',
    'AMT_CREDIT_MAX_OVERDUE_MAX',
    'DAYS_ENDDATE_MIN',
    'DEBT_TO_CREDIT_SUM_MAX',
    'DAYS_ENDDATE_MEAN_CUSTOM',
    'CREDIT_SUM_RATIO_WEIGHTED',
    'CREDIT_SUM_RATIO',
    'DAYS_ENDDATE_DIFF_STD',
    'ENDDATE_DIFF_MEAN',
    'AMT_ANNUITY_POS_ENDDATE_SUM',
    'AMT_CREDIT_SUM_LIMIT_MAX',
    'DAYS_ENDDATE_MEDIAN_CUSTOM',
    'DAYS_CREDIT_MEDIAN',
    'DAYS_CREDIT_MIN',
    'DAYS_CREDIT_MAX'
]

In [ ]:
bureau_train = bureau_train[to_keep + ['TARGET']]
bureau_test = bureau_test[to_keep + ['TARGET']]

In [ ]:
bureau_train.head()

,SK_ID_CURR,DAYS_ENDDATE_FACT_CREDIT_DIFF_MAX,DAYS_ENDDATE_FACT_CREDIT_DIFF_MEAN,DAYS_ENDDATE_MAX,CREDIT_UTILIZATION_MEAN,CREDIT_UTILIZATION_MEDIAN,DAYS_ENDDATE_FACT_CREDIT_DIFF_MEDIAN,AMT_CREDIT_MAX_OVERDUE_MEDIAN,DEBT_TO_CREDIT_SUM_MEDIAN,DEBT_TO_CREDIT_SUM_MEAN,...,CREDIT_SUM_RATIO,DAYS_ENDDATE_DIFF_STD,ENDDATE_DIFF_MEAN,AMT_ANNUITY_POS_ENDDATE_SUM,AMT_CREDIT_SUM_LIMIT_MAX,DAYS_ENDDATE_MEDIAN_CUSTOM,DAYS_CREDIT_MEDIAN,DAYS_CREDIT_MIN,DAYS_CREDIT_MAX,TARGET
1,100002,609.0,277.000000,780.0,0.0,0.0,231.0,40.5,0.000000,0.136545,...,-4.547563,437.954678,252.600000,0.0,31988.565,-424.5,-1042.5,-103,-1437,0
2,100003,1096.0,568.333333,1216.0,0.0,0.0,455.0,0.0,0.000000,0.000000,...,0.908956,253.714406,-34.000000,0.0,810000.000,-480.0,-1205.5,-606,-2586,0
4,100005,250.0,250.000000,1324.0,NaN,NaN,250.0,0.0,0.848974,0.601256,...,2.893261,NaN,-5.000000,4261.5,0.000,122.0,-137.0,-62,-373,0
5,100007,366.0,366.000000,-783.0,NaN,NaN,366.0,0.0,0.000000,0.000000,...,-0.405280,NaN,0.000000,0.0,0.000,0.0,-1149.0,-1149,-1149,0
7,100009,893.0,357.214286,1402.0,NaN,NaN,244.5,0.0,0.000000,0.169369,...,-0.488184,509.841028,114.785714,0.0,0.000,-1068.0,-1276.0,-239,-2882,1


In [ ]:
bureau_test.head()

,SK_ID_CURR,DAYS_ENDDATE_FACT_CREDIT_DIFF_MAX,DAYS_ENDDATE_FACT_CREDIT_DIFF_MEAN,DAYS_ENDDATE_MAX,CREDIT_UTILIZATION_MEAN,CREDIT_UTILIZATION_MEDIAN,DAYS_ENDDATE_FACT_CREDIT_DIFF_MEDIAN,AMT_CREDIT_MAX_OVERDUE_MEDIAN,DEBT_TO_CREDIT_SUM_MEDIAN,DEBT_TO_CREDIT_SUM_MEAN,...,CREDIT_SUM_RATIO,DAYS_ENDDATE_DIFF_STD,ENDDATE_DIFF_MEAN,AMT_ANNUITY_POS_ENDDATE_SUM,AMT_CREDIT_SUM_LIMIT_MAX,DAYS_ENDDATE_MEDIAN_CUSTOM,DAYS_CREDIT_MEDIAN,DAYS_CREDIT_MIN,DAYS_CREDIT_MAX,TARGET
0,100001,335.0,228.75,1778.0,NaN,NaN,274.0,NaN,0.000000,0.282518,...,6.818219,334.957709,197.0,24817.5,0.0,-179.0,-857.0,-49,-1572,0
3,100004,643.0,334.50,-382.0,NaN,NaN,334.5,0.0,0.000000,0.000000,...,-0.793279,62.225397,44.0,0.0,0.0,0.0,-867.0,-408,-1326,0
6,100008,307.0,188.00,471.0,NaN,NaN,188.0,0.0,0.000000,0.299018,...,0.341768,125.157900,86.5,0.0,0.0,-792.0,-1097.0,-78,-1097,1
15,100019,NaN,NaN,10463.0,0.0,0.0,NaN,0.0,0.272745,0.272745,...,-4.463499,NaN,NaN,54000.0,270000.0,5441.0,-495.0,-495,-495,0
23,100029,742.0,311.00,1563.0,NaN,NaN,110.0,NaN,0.000000,0.328629,...,-2.223814,2097.560249,1205.0,0.0,0.0,-1286.0,-1375.5,-110,-2806,0


In [ ]:
bureau_train.to_parquet('BUREAU_TRAIN.parquet')
bureau_test.to_parquet('BUREAU_TEST.parquet')

# Behavioural Branch

## Installments

In [ ]:
installments_payments.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In our dataset, `installments_payments` does not have many features. Despite that, there is still a bit of feature engineering we should do. First of all, the values in the last four columns do not tell anything by themselves, so which should transform to show payment amount and date relative to the planned amounts. Second, there exists a problem of aggregating the results

In [ ]:
installments_payments.shape

(13605401, 8)

In [ ]:
installments_payments['SK_ID_PREV'].nunique()

997752

In [ ]:
installments_payments['SK_ID_CURR'].nunique()

339587

As seen above, we have a lot to aggregate, and it may not be as simple to decide on what to actually do. The best idea in this case is going to be simple time-aware aggregation with feature engineering. The main reason why Deep Learning would not work here is, while there are clients who have taken a large quantity of loans in the past, for the overwhelming majority this number would be below 5, and it would be hard for GRU, the model we would most likely use, to learn the patterns from it. Keeping that in mind, let's get to work

In [ ]:
installments_payments.dtypes

SK_ID_PREV                  int64
SK_ID_CURR                  int64
NUM_INSTALMENT_VERSION    float64
NUM_INSTALMENT_NUMBER       int64
DAYS_INSTALMENT           float64
DAYS_ENTRY_PAYMENT        float64
AMT_INSTALMENT            float64
AMT_PAYMENT               float64
dtype: object

And now for the engineering part

In [ ]:
installments_payments.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [ ]:
installments_payments["PAYMENT_DIFF"] = np.where(
    installments_payments['DAYS_ENTRY_PAYMENT'].isna(),
    np.nan,
    np.where(
        installments_payments['DAYS_ENTRY_PAYMENT'] < installments_payments['DAYS_INSTALMENT'],
        -1,
        np.where(
            installments_payments['DAYS_ENTRY_PAYMENT'] > installments_payments['DAYS_INSTALMENT'],
            np.abs(installments_payments['DAYS_INSTALMENT'] - installments_payments['DAYS_ENTRY_PAYMENT']),
            0
        )
    )
)
installments_payments["AMT_DIFF"] = installments_payments["AMT_INSTALMENT"] - installments_payments["AMT_PAYMENT"]

installments_payments["AMT_DIFF_PCT"] = (
    (installments_payments["AMT_INSTALMENT"] - installments_payments["AMT_PAYMENT"])
    / installments_payments["AMT_INSTALMENT"].replace(0, np.nan)
)

installments_payments["IS_LATE"] = (installments_payments["DAYS_ENTRY_PAYMENT"] > installments_payments["DAYS_INSTALMENT"]).astype(int)
installments_payments["IS_EARLY"] = (installments_payments["DAYS_ENTRY_PAYMENT"] < installments_payments["DAYS_INSTALMENT"]).astype(int)
installments_payments["PAID_ON_TIME"] = (installments_payments["DAYS_ENTRY_PAYMENT"] - installments_payments["DAYS_INSTALMENT"]).abs().le(7).astype(int)

installments_payments["OVERPAID"] = (installments_payments["AMT_PAYMENT"] > installments_payments["AMT_INSTALMENT"]).astype(int)
installments_payments["UNDERPAID"] = (installments_payments["AMT_PAYMENT"] < installments_payments["AMT_INSTALMENT"]).astype(int)
installments_payments["PAID_ZERO"] = (installments_payments["AMT_PAYMENT"] == 0).astype(int)

def compute_on_time_streak(group):
    streak = []
    current_streak = 0
    for val in group:
        if val == 0:
            current_streak += 1
        else:
            current_streak = 0
        streak.append(current_streak)
    return streak

installments_payments["PAYMENT_STREAK"] = (
    installments_payments
    .sort_values(["SK_ID_PREV", "DAYS_INSTALMENT"])
    .groupby("SK_ID_PREV")["IS_LATE"]
    .transform(compute_on_time_streak)
)

installments_payments['DAYS_LATE_PAYMENT'] = np.where(
    installments_payments['DAYS_ENTRY_PAYMENT'].isna(),
    np.nan,
    (installments_payments['DAYS_ENTRY_PAYMENT'] - installments_payments['DAYS_INSTALMENT']).apply(lambda x: max(0, x))
)

installments_payments['DAYS_EARLY_PAYMENT'] = np.where(
    installments_payments['DAYS_ENTRY_PAYMENT'].isna(),
    np.nan,
    (installments_payments['DAYS_INSTALMENT'] - installments_payments['DAYS_ENTRY_PAYMENT']).apply(lambda x: max(0, x))
)

installments_payments['PAYMENT_RATIO'] = np.where(
    installments_payments['AMT_INSTALMENT'] == 0,
    np.nan,
    installments_payments['AMT_PAYMENT'] / installments_payments['AMT_INSTALMENT']
)

installments_payments = installments_payments.sort_values(by=['SK_ID_PREV', 'NUM_INSTALMENT_NUMBER'])
installments_payments['INSTALLMENT_VERSION_CHANGED'] = (
    installments_payments.groupby('SK_ID_PREV')['NUM_INSTALMENT_VERSION'].diff() != 0
).astype(int)
installments_payments['INSTALLMENT_VERSION_CHANGED'] = installments_payments['INSTALLMENT_VERSION_CHANGED'].fillna(0).astype(int)

def compute_late_streak(group):
    streak = []
    current_streak = 0
    for val in group:
        if val == 1:
            current_streak += 1
        else:
            current_streak = 0
        streak.append(current_streak)
    return streak

installments_payments["CONSECUTIVE_LATE_STREAK_CURRENT"] = (
    installments_payments
    .sort_values(["SK_ID_PREV", "DAYS_INSTALMENT"])
    .groupby("SK_ID_PREV")["IS_LATE"]
    .transform(compute_late_streak)
)

max_installment_per_loan = installments_payments.groupby('SK_ID_PREV')['NUM_INSTALMENT_NUMBER'].transform('max')
installments_payments['LAST_INSTALLMENT_FOR_LOAN'] = (
    installments_payments['NUM_INSTALMENT_NUMBER'] == max_installment_per_loan
).astype(int)

installments_payments = installments_payments.sort_values(by=['SK_ID_PREV', 'DAYS_INSTALMENT'])
installments_payments['TIME_TO_NEXT_INSTALLMENT_SCHEDULED'] = installments_payments.groupby('SK_ID_PREV')['DAYS_INSTALMENT'].diff(-1).abs()

installments_payments = installments_payments.sort_values(by=['SK_ID_PREV', 'DAYS_ENTRY_PAYMENT'])
installments_payments['TIME_TO_NEXT_PAYMENT_ACTUAL'] = installments_payments.groupby('SK_ID_PREV')['DAYS_ENTRY_PAYMENT'].diff(-1).abs()


In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
installments_payments.drop(columns=['SK_ID_PREV', 'NUM_INSTALMENT_VERSION', 'NUM_INSTALMENT_NUMBER', 'DAYS_INSTALMENT', 'AMT_INSTALMENT', 'AMT_PAYMENT'], inplace=True, errors='ignore')
installments_payments.head()

,SK_ID_CURR,DAYS_ENTRY_PAYMENT,PAYMENT_DIFF,AMT_DIFF,AMT_DIFF_PCT,IS_LATE,IS_EARLY,PAID_ON_TIME,OVERPAID,UNDERPAID,PAID_ZERO,PAYMENT_STREAK,DAYS_LATE_PAYMENT,DAYS_EARLY_PAYMENT,PAYMENT_RATIO,INSTALLMENT_VERSION_CHANGED,CONSECUTIVE_LATE_STREAK_CURRENT,LAST_INSTALLMENT_FOR_LOAN,TIME_TO_NEXT_INSTALLMENT_SCHEDULED,TIME_TO_NEXT_PAYMENT_ACTUAL
512588,158271,-294.0,-1.0,0.0,0.0,0,1,0,0,0,0,1,0.0,26.0,1.0,1,0,0,30.0,50.0
2159480,158271,-244.0,-1.0,0.0,0.0,0,1,1,0,0,0,2,0.0,6.0,1.0,1,0,1,NaN,NaN
1214732,101962,-1611.0,-1.0,0.0,0.0,0,1,0,0,0,0,1,0.0,11.0,1.0,1,0,0,30.0,36.0
1631862,101962,-1575.0,-1.0,0.0,0.0,0,1,1,0,0,0,2,0.0,5.0,1.0,0,0,0,30.0,16.0
3411021,101962,-1559.0,-1.0,0.0,0.0,0,1,0,0,0,0,3,0.0,19.0,1.0,0,0,0,30.0,5.0


Before we proceed any further, a thing that we should do here is aggregation. Here it would be unwise to use only means or medians, as the features may have vastly different weight due to some being older than others. It is a rational assumption, as people financial situation may change over time. For that reason, when aggregating, we are going to use weighted aggregations AND medians or modes

In [ ]:
installments_payments.to_parquet('INSTALLMENTS_BEFORE_AGG.parquet', index=False)

In [ ]:
installments_payments = pd.read_parquet('INSTALLMENTS_BEFORE_AGG.parquet')

In [ ]:
features_to_aggregate = [
    'PAYMENT_DIFF',
    'AMT_DIFF',
    'AMT_DIFF_PCT',
    'IS_LATE',
    'IS_EARLY',
    'PAID_ON_TIME',
    'OVERPAID',
    'UNDERPAID',
    'PAID_ZERO',
    'PAYMENT_STREAK',
    'DAYS_LATE_PAYMENT',
    'DAYS_EARLY_PAYMENT',
    'PAYMENT_RATIO',
    'INSTALLMENT_VERSION_CHANGED',
    'CONSECUTIVE_LATE_STREAK_CURRENT',
    'LAST_INSTALLMENT_FOR_LOAN',
    'TIME_TO_NEXT_INSTALLMENT_SCHEDULED',
    'TIME_TO_NEXT_PAYMENT_ACTUAL'
]

In [ ]:
installments_agg = pd.DataFrame(index=installments_payments['SK_ID_CURR'].unique()).sort_index()

basic_agg_df = installments_payments.groupby('SK_ID_CURR')[features_to_aggregate].mean()
for col in features_to_aggregate:
    installments_agg[f'{col}_basic'] = basic_agg_df[col]


recency_coefficients = [0.001, 0.005, 0.01, 0.05]

for coeff in recency_coefficients:
    temp_df = installments_payments.copy()

    temp_df['RECENCY_WEIGHT'] = np.exp(coeff * temp_df['DAYS_ENTRY_PAYMENT'])

    weight_sums = temp_df.groupby('SK_ID_CURR')['RECENCY_WEIGHT'].sum()

    for col in features_to_aggregate:
        weighted_feature_sum = (temp_df[col].fillna(0) * temp_df['RECENCY_WEIGHT']).groupby(temp_df['SK_ID_CURR']).sum()

        weighted_avg = np.where(weight_sums == 0, np.nan, weighted_feature_sum / weight_sums)

        col_name_suffix = str(coeff).replace(".", "")
        installments_agg[f'{col}_{col_name_suffix}'] = weighted_avg

In [ ]:
installments_agg = installments_agg.reset_index()
installments_agg.rename(columns={'index': 'SK_ID_CURR'}, inplace=True)
installments_agg.head()

,SK_ID_CURR,PAYMENT_DIFF_basic,AMT_DIFF_basic,AMT_DIFF_PCT_basic,IS_LATE_basic,IS_EARLY_basic,PAID_ON_TIME_basic,OVERPAID_basic,UNDERPAID_basic,PAID_ZERO_basic,...,PAID_ZERO_005,PAYMENT_STREAK_005,DAYS_LATE_PAYMENT_005,DAYS_EARLY_PAYMENT_005,PAYMENT_RATIO_005,INSTALLMENT_VERSION_CHANGED_005,CONSECUTIVE_LATE_STREAK_CURRENT_005,LAST_INSTALLMENT_FOR_LOAN_005,TIME_TO_NEXT_INSTALLMENT_SCHEDULED_005,TIME_TO_NEXT_PAYMENT_ACTUAL_005
0,100001,1.000000,0.0,0.0,0.142857,0.571429,0.428571,0.0,0.0,0.0,...,0.0,3.782986,7.482313e-27,9.581209,1.0,0.825037,6.802103e-28,0.814525,5.564265,5.840600
1,100002,-1.000000,0.0,0.0,0.000000,1.000000,0.000000,0.0,0.0,0.0,...,0.0,18.578268,0.000000e+00,20.135506,1.0,0.662928,0.000000e+00,0.662928,10.112165,7.020502
2,100003,-1.000000,0.0,0.0,0.000000,1.000000,0.520000,0.0,0.0,0.0,...,0.0,6.662221,0.000000e+00,6.970197,1.0,0.738079,0.000000e+00,0.740086,7.797422,6.985358
3,100004,-1.000000,0.0,0.0,0.000000,1.000000,0.333333,0.0,0.0,0.0,...,0.0,2.806415,0.000000e+00,4.050145,1.0,0.862098,0.000000e+00,0.834256,4.972306,5.855400
4,100005,-0.777778,0.0,0.0,0.111111,0.888889,0.222222,0.0,0.0,0.0,...,0.0,3.861263,2.826096e-03,5.784176,1.0,0.887935,2.826096e-03,0.887933,3.362007,4.811971


In [ ]:
installments_agg.shape

(339587, 91)

In [ ]:
installments_train = installments_agg[
    installments_agg['SK_ID_CURR'].isin(set(y_train.index))
].copy()

installments_test = installments_agg[
    installments_agg['SK_ID_CURR'].isin(set(y_test.index))
].copy()

installments_train['TARGET'] = installments_train['SK_ID_CURR'].map(y_train)
installments_test['TARGET'] = installments_test['SK_ID_CURR'].map(y_test)

And now we can do actual testing. Again, I know that here the result is not optimal, as we haven't aggregated the features yet, but it would still be useful to know what here is actually useful

In [ ]:
categorical_cols = installments_train.select_dtypes(include=["category", "object", "bool"]).columns.tolist()
categorical_cols = [col for col in categorical_cols if col not in ('TARGET', 'SK_ID_PREV', 'SK_ID_CURR')]
continuous_cols = installments_train.select_dtypes(include=["float", "int"]).columns.tolist()
continuous_cols = [col for col in continuous_cols if col not in ('TARGET', 'SK_ID_PREV', 'SK_ID_CURR')]

### Mutual Information

In [ ]:
pd.set_option('display.max_rows', None)
sf.compare_mi(installments_train[continuous_cols + ['TARGET']], 'TARGET')

/home/ilyac/Desktop/Programming/Credit/support_file.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([results_df, new_result], ignore_index=True)
/home/ilyac/Desktop/Programming/Credit/.venv/lib/python3.10/site-packages/sklearn/metrics/cluster/_supervised.py:49: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  type_label = type_of_target(labels_true)
/home/ilyac/Desktop/Programming/Credit/.venv/lib/python3.10/site-packages/sklearn/metrics/cluster/_supervised.py:49: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  type_label = type_of_target(labels_true)
/home/ilyac/Desktop/Programming/Credit/.venv/lib/python3.10/site-packages/sklearn/metri

,Feature,Score
0,DAYS_EARLY_PAYMENT_001,0.278879
1,DAYS_EARLY_PAYMENT_0005,0.278825
2,DAYS_EARLY_PAYMENT_0001,0.278777
3,TIME_TO_NEXT_INSTALLMENT_SCHEDULED_001,0.277920
4,LAST_INSTALLMENT_FOR_LOAN_001,0.277909
5,TIME_TO_NEXT_PAYMENT_ACTUAL_001,0.277893
6,TIME_TO_NEXT_INSTALLMENT_SCHEDULED_0005,0.277546
7,TIME_TO_NEXT_PAYMENT_ACTUAL_0005,0.277478
8,PAYMENT_STREAK_001,0.277474
9,LAST_INSTALLMENT_FOR_LOAN_0005,0.277471


Wow, thats a massive success. We have a lot of features here to choose from

### XGBoost Feature Importance

In [ ]:
sf.compare_xgb(installments_train[continuous_cols + ['TARGET']], 'TARGET')

/home/ilyac/Desktop/Programming/Credit/support_file.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([results_df, new_result], ignore_index=True)


,Feature,Score
0,TIME_TO_NEXT_INSTALLMENT_SCHEDULED_0005,5.268462e-02
1,INSTALLMENT_VERSION_CHANGED_001,4.052950e-02
2,TIME_TO_NEXT_PAYMENT_ACTUAL_basic,3.379776e-02
3,TIME_TO_NEXT_INSTALLMENT_SCHEDULED_001,3.364395e-02
4,DAYS_EARLY_PAYMENT_0001,3.298979e-02
5,DAYS_EARLY_PAYMENT_001,3.119397e-02
6,DAYS_EARLY_PAYMENT_basic,3.097472e-02
7,AMT_DIFF_basic,2.773943e-02
8,INSTALLMENT_VERSION_CHANGED_0001,2.676151e-02
9,TIME_TO_NEXT_INSTALLMENT_SCHEDULED_0001,2.453494e-02


Here we can see some values that are really high for XGB and this dataset, which is of course a good sign

In [ ]:
to_keep = [
    'SK_ID_CURR',
    'DAYS_EARLY_PAYMENT_001',
    'TIME_TO_NEXT_INSTALLMENT_SCHEDULED_001',
    'LAST_INSTALLMENT_FOR_LOAN_001',
    'TIME_TO_NEXT_PAYMENT_ACTUAL_001',
    'PAYMENT_STREAK_001',
    'INSTALLMENT_VERSION_CHANGED_001',
    'PAID_ON_TIME_001',
    'AMT_DIFF_basic',
    'PAYMENT_RATIO_001',
    'AMT_DIFF_PCT_0005'
]

At this point I owe the reader an explanation of what's happened. First, let's talk about the approach I took. As already mentioned, here we have an aggregation where the rows are weighed by recency. In additional to the 'basic' aggregation, instead of relying on one value for the coefficient of the exponential here, we tested multiple, and, after cross-referencing two tables, we narrowed down the list to only a handful of features. The reason for there being less tests here is that some tests, notably 'Permutation Importance', are extremely memory-heavy, and, having 13 million rows, we might use other test. Additionally, if you take a look at the `to_keep` list above, you would notice that there are very few features which actually passed the selection process. This is because, while the lists of good results were massive, it was like that only because instances like `feature1_basic`, `feature1_001`, `feature1_0005` have all made it, and I had to manually select the good ones. The 'version' of the feature, meaning what coefficient has been used for the recency-based weighing, selected here is one that produced better results, so higher is better

In [ ]:
installments_train = installments_train[to_keep + ['TARGET']]
installments_test = installments_test[to_keep + ['TARGET']]

In [ ]:
installments_train.to_parquet('INSTALLMENTS_TRAIN.parquet')
installments_test.to_parquet('INSTALLMENTS_TEST.parquet')

## Credit Card Balance

In [ ]:
credit_card_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [ ]:
credit_card_balance.shape

(3840312, 23)

In [ ]:
credit_card_balance.dtypes

SK_ID_PREV                      int64
SK_ID_CURR                      int64
MONTHS_BALANCE                  int64
AMT_BALANCE                   float64
AMT_CREDIT_LIMIT_ACTUAL         int64
AMT_DRAWINGS_ATM_CURRENT      float64
AMT_DRAWINGS_CURRENT          float64
AMT_DRAWINGS_OTHER_CURRENT    float64
AMT_DRAWINGS_POS_CURRENT      float64
AMT_INST_MIN_REGULARITY       float64
AMT_PAYMENT_CURRENT           float64
AMT_PAYMENT_TOTAL_CURRENT     float64
AMT_RECEIVABLE_PRINCIPAL      float64
AMT_RECIVABLE                 float64
AMT_TOTAL_RECEIVABLE          float64
CNT_DRAWINGS_ATM_CURRENT      float64
CNT_DRAWINGS_CURRENT            int64
CNT_DRAWINGS_OTHER_CURRENT    float64
CNT_DRAWINGS_POS_CURRENT      float64
CNT_INSTALMENT_MATURE_CUM     float64
NAME_CONTRACT_STATUS           object
SK_DPD                          int64
SK_DPD_DEF                      int64
dtype: object

Even though the length of the dataframe is not as big as the previous ones are, there are a lot of features for us to test out. The general principles that we are going to follow are going to be exactly the same as with "Installments". There is not much for us to 'correct' here for us, as, after closely inspecting the data, the datatypes and formatting are correct, so we will go straight to feature engineering

An additional piece of information worth mentioning is that this dataframe is closely related to the "Installments" table, as those are stats for credit card use of the clients. While I considered merging the two, I believe there is no need to do it as the most important features such as `AMT_INST_MIN_REGULARITY` and `AMT_RECEIVABLE_PRINCIPAL` (which isn't even explicitly stated in the previous dataframe, interestingly) are here

In [ ]:
for col in ['AMT_BALANCE', 'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_ATM_CURRENT',
            'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT', 'AMT_DRAWINGS_POS_CURRENT',
            'AMT_INST_MIN_REGULARITY', 'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT',
            'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE',
            'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT',
            'CNT_DRAWINGS_POS_CURRENT', 'CNT_INSTALMENT_MATURE_CUM', 'SK_DPD', 'SK_DPD_DEF']:
    credit_card_balance[col] = pd.to_numeric(credit_card_balance[col], errors='coerce')


credit_card_balance['BALANCE_TO_LIMIT_RATIO'] = (
    credit_card_balance['AMT_BALANCE'] / credit_card_balance['AMT_CREDIT_LIMIT_ACTUAL'].replace(0, np.nan)
)

credit_card_balance['BALANCE_TO_MIN_REG_RATIO'] = (
    credit_card_balance['AMT_BALANCE'] / credit_card_balance['AMT_INST_MIN_REGULARITY'].replace(0, np.nan)
)


credit_card_balance['CURRENT_PAYMENT_BELOW_MIN_REG'] = (
    credit_card_balance['AMT_PAYMENT_CURRENT'] < credit_card_balance['AMT_INST_MIN_REGULARITY']
).astype(float)
credit_card_balance['CURRENT_PAYMENT_BELOW_MIN_REG'] = credit_card_balance['CURRENT_PAYMENT_BELOW_MIN_REG'].fillna(0).astype(int)

credit_card_balance['CURRENT_PAYMENT_MIN_REG_DIFF_PCT'] = (
    (credit_card_balance['AMT_PAYMENT_CURRENT'] - credit_card_balance['AMT_INST_MIN_REGULARITY']) /
    credit_card_balance['AMT_INST_MIN_REGULARITY'].replace(0, np.nan)
)

credit_card_balance['TOTAL_PAYMENT_BELOW_MIN_REG'] = (
    credit_card_balance['AMT_PAYMENT_TOTAL_CURRENT'] < credit_card_balance['AMT_INST_MIN_REGULARITY']
).astype(float)
credit_card_balance['TOTAL_PAYMENT_BELOW_MIN_REG'] = credit_card_balance['TOTAL_PAYMENT_BELOW_MIN_REG'].fillna(0).astype(int)


credit_card_balance['TOTAL_PAYMENT_MIN_REG_DIFF_PCT'] = (
    (credit_card_balance['AMT_PAYMENT_TOTAL_CURRENT'] - credit_card_balance['AMT_INST_MIN_REGULARITY']) /
    credit_card_balance['AMT_INST_MIN_REGULARITY'].replace(0, np.nan)
)

def safe_division(numerator, denominator):
    return numerator / denominator.replace(0, np.nan)

credit_card_balance['DRAWINGS_ATM_PCT'] = safe_division(
    credit_card_balance['AMT_DRAWINGS_ATM_CURRENT'], credit_card_balance['AMT_DRAWINGS_CURRENT']
)
credit_card_balance['DRAWINGS_POS_PCT'] = safe_division(
    credit_card_balance['AMT_DRAWINGS_POS_CURRENT'], credit_card_balance['AMT_DRAWINGS_CURRENT']
)
credit_card_balance['DRAWINGS_OTHER_PCT'] = safe_division(
    credit_card_balance['AMT_DRAWINGS_OTHER_CURRENT'], credit_card_balance['AMT_DRAWINGS_CURRENT']
)
credit_card_balance['DRAWINGS_TO_LIMIT_RATIO'] = safe_division(
    credit_card_balance['AMT_DRAWINGS_CURRENT'], credit_card_balance['AMT_CREDIT_LIMIT_ACTUAL']
)


credit_card_balance['CNT_DRAWINGS_ATM_PCT'] = safe_division(
    credit_card_balance['CNT_DRAWINGS_ATM_CURRENT'], credit_card_balance['CNT_DRAWINGS_CURRENT']
)
credit_card_balance['CNT_DRAWINGS_POS_PCT'] = safe_division(
    credit_card_balance['CNT_DRAWINGS_POS_CURRENT'], credit_card_balance['CNT_DRAWINGS_CURRENT']
)
credit_card_balance['CNT_DRAWINGS_OTHER_PCT'] = safe_division(
    credit_card_balance['CNT_DRAWINGS_OTHER_CURRENT'], credit_card_balance['CNT_DRAWINGS_CURRENT']
)

credit_card_balance['CNT_DRAWINGS_TO_LIMIT_RATIO'] = safe_division(
    credit_card_balance['CNT_DRAWINGS_CURRENT'], credit_card_balance['AMT_CREDIT_LIMIT_ACTUAL']
)


credit_card_balance['HAS_ATM_DRAWINGS'] = (credit_card_balance['CNT_DRAWINGS_ATM_CURRENT'] > 0).astype(int)
credit_card_balance['HAS_POS_DRAWINGS'] = (credit_card_balance['CNT_DRAWINGS_POS_CURRENT'] > 0).astype(int)
credit_card_balance['HAS_OTHER_DRAWINGS'] = (credit_card_balance['CNT_DRAWINGS_OTHER_CURRENT'] > 0).astype(int)
credit_card_balance['HAS_ANY_DRAWINGS'] = (credit_card_balance['CNT_DRAWINGS_CURRENT'] > 0).astype(int)


credit_card_balance['PRINCIPAL_TO_TOTAL_RECEIVABLE_RATIO'] = safe_division(
    credit_card_balance['AMT_RECEIVABLE_PRINCIPAL'], credit_card_balance['AMT_TOTAL_RECEIVABLE']
)
credit_card_balance['RECEIVABLE_TO_LIMIT_RATIO'] = safe_division(
    credit_card_balance['AMT_TOTAL_RECEIVABLE'], credit_card_balance['AMT_CREDIT_LIMIT_ACTUAL']
)
credit_card_balance['BALANCE_TO_TOTAL_RECEIVABLE_RATIO'] = safe_division(
    credit_card_balance['AMT_BALANCE'], credit_card_balance['AMT_TOTAL_RECEIVABLE']
)

credit_card_balance['PAYMENT_TO_TOTAL_RECEIVABLE_RATIO'] = safe_division(
    credit_card_balance['AMT_PAYMENT_TOTAL_CURRENT'], credit_card_balance['AMT_TOTAL_RECEIVABLE']
)
credit_card_balance['PAYMENT_TO_PRINCIPAL_RECEIVABLE_RATIO'] = safe_division(
    credit_card_balance['AMT_PAYMENT_TOTAL_CURRENT'], credit_card_balance['AMT_RECEIVABLE_PRINCIPAL']
)


credit_card_balance['IS_DPD'] = (credit_card_balance['SK_DPD'] > 0).astype(int)
credit_card_balance['IS_DPD_DEF'] = (credit_card_balance['SK_DPD_DEF'] > 0).astype(int)
credit_card_balance['IS_SEVERE_DPD'] = (credit_card_balance['SK_DPD'] >= 30).astype(int)

credit_card_balance['DPD_TOLERANCE_DIFF'] = credit_card_balance['SK_DPD'] - credit_card_balance['SK_DPD_DEF']

credit_card_balance['MIN_REG_TO_LIMIT_RATIO'] = safe_division(
    credit_card_balance['AMT_INST_MIN_REGULARITY'], credit_card_balance['AMT_CREDIT_LIMIT_ACTUAL']
)
credit_card_balance['PAYMENT_COVERAGE_OF_BALANCE'] = safe_division(
    credit_card_balance['AMT_PAYMENT_TOTAL_CURRENT'], credit_card_balance['AMT_BALANCE']
)
credit_card_balance['PAYMENT_COVERAGE_OF_PRINCIPAL'] = safe_division(
    credit_card_balance['AMT_PAYMENT_TOTAL_CURRENT'], credit_card_balance['AMT_RECEIVABLE_PRINCIPAL']
)

credit_card_balance['OVERLIMIT_FLAG'] = (
    (credit_card_balance['AMT_BALANCE'] > credit_card_balance['AMT_CREDIT_LIMIT_ACTUAL'])
    .astype(int)
)
credit_card_balance['BALANCE_OVER_LIMIT_AMOUNT'] = np.where(
    credit_card_balance['AMT_BALANCE'] > credit_card_balance['AMT_CREDIT_LIMIT_ACTUAL'],
    credit_card_balance['AMT_BALANCE'] - credit_card_balance['AMT_CREDIT_LIMIT_ACTUAL'],
    0
)

credit_card_balance['DRAWINGS_PER_COUNT'] = safe_division(
    credit_card_balance['AMT_DRAWINGS_CURRENT'], credit_card_balance['CNT_DRAWINGS_CURRENT']
)

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
credit_card_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,BALANCE_TO_LIMIT_RATIO,BALANCE_TO_MIN_REG_RATIO,CURRENT_PAYMENT_BELOW_MIN_REG,CURRENT_PAYMENT_MIN_REG_DIFF_PCT,TOTAL_PAYMENT_BELOW_MIN_REG,TOTAL_PAYMENT_MIN_REG_DIFF_PCT,DRAWINGS_ATM_PCT,DRAWINGS_POS_PCT,DRAWINGS_OTHER_PCT,DRAWINGS_TO_LIMIT_RATIO,CNT_DRAWINGS_ATM_PCT,CNT_DRAWINGS_POS_PCT,CNT_DRAWINGS_OTHER_PCT,CNT_DRAWINGS_TO_LIMIT_RATIO,HAS_ATM_DRAWINGS,HAS_POS_DRAWINGS,HAS_OTHER_DRAWINGS,HAS_ANY_DRAWINGS,PRINCIPAL_TO_TOTAL_RECEIVABLE_RATIO,RECEIVABLE_TO_LIMIT_RATIO,BALANCE_TO_TOTAL_RECEIVABLE_RATIO,PAYMENT_TO_TOTAL_RECEIVABLE_RATIO,PAYMENT_TO_PRINCIPAL_RECEIVABLE_RATIO,IS_DPD,IS_DPD_DEF,IS_SEVERE_DPD,DPD_TOLERANCE_DIFF,MIN_REG_TO_LIMIT_RATIO,PAYMENT_COVERAGE_OF_BALANCE,PAYMENT_COVERAGE_OF_PRINCIPAL,OVERLIMIT_FLAG,BALANCE_OVER_LIMIT_AMOUNT,DRAWINGS_PER_COUNT
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,1800.0,0.000,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0,0.000422,0.033505,0,0.058621,0,0.058621,0.0,1.0,0.0,0.00650,0.0,1.0,0.0,0.000007,0,1,0,1,NaN,0.000000,NaN,NaN,NaN,0,0,0,0,0.012595,31.595577,NaN,0,0.000,877.5
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,2250.0,60175.080,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0,1.421679,28.433580,0,0.000000,0,0.000000,1.0,0.0,0.0,0.05000,1.0,0.0,0.0,0.000022,1,0,0,1,0.927546,1.441679,0.986127,0.034682,0.037391,0,0,0,0,0.050000,0.035170,0.037391,1,18975.555,2250.0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.0,2250.0,26926.425,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0,0.070700,14.140100,0,0.000000,0,0.000000,NaN,NaN,NaN,0.00000,NaN,NaN,NaN,0.000000,0,0,0,0,0.855892,0.069911,1.011289,0.071519,0.083561,0,0,0,0,0.005000,0.070721,0.083561,0,0.000,NaN
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.0,11925.0,224949.285,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0,1.051432,20.055690,0,0.010956,0,0.010956,1.0,0.0,0.0,0.01000,1.0,0.0,0.0,0.000004,1,0,0,1,0.965245,1.035773,1.015118,0.051170,0.053012,0,0,0,0,0.052426,0.050407,0.053012,1,11572.110,2250.0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.0,27000.0,443044.395,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0,1.008710,19.800289,0,0.177759,0,0.177759,0.0,1.0,0.0,0.02566,0.0,1.0,0.0,0.000002,0,1,0,1,0.976042,1.008710,1.000000,0.059482,0.060942,0,0,0,0,0.050944,0.059482,0.060942,1,3919.455,11547.0


Extremely amazed by 1.5s execution time, we continue with our analysis. Now that we have engineered a large amount of various features, we will have to aggregate those and determine how good of a predictor those features are. We are going to do it in the same way as the last time, using means and mode and a coefficient for how long ago something was

In [ ]:
for col in credit_card_balance.columns:
    if credit_card_balance[col].dtype == 'object' and col != 'NAME_CONTRACT_STATUS':
        credit_card_balance[col] = pd.to_numeric(credit_card_balance[col], errors='coerce')



numerical_features = credit_card_balance.select_dtypes(include=np.number).columns.tolist()
numerical_features.remove('SK_ID_PREV')
numerical_features.remove('SK_ID_CURR')
numerical_features.remove('MONTHS_BALANCE')

categorical_features = ['NAME_CONTRACT_STATUS']

def most_frequent_with_tiebreaker(series, global_freq):
    top = series.value_counts().nlargest(2)
    if len(top) == 1:
        return top.index[0]
    top_candidates = [val for val in top.index if val in global_freq.index]
    if not top_candidates:
        return np.nan
    return global_freq.loc[top_candidates].idxmax()

contract_status_global_freq = credit_card_balance['NAME_CONTRACT_STATUS'].value_counts()


credit_card_balance_agg = pd.DataFrame(index=credit_card_balance['SK_ID_CURR'].unique()).sort_index()

basic_agg_numerical_df = credit_card_balance.groupby('SK_ID_CURR')[numerical_features].mean()
for col in numerical_features:
    credit_card_balance_agg[f'{col}_basic'] = basic_agg_numerical_df[col]


basic_agg_categorical_series = credit_card_balance.groupby('SK_ID_CURR')['NAME_CONTRACT_STATUS'].apply(
    lambda s: most_frequent_with_tiebreaker(s, contract_status_global_freq)
)
credit_card_balance_agg[f'NAME_CONTRACT_STATUS_basic'] = basic_agg_categorical_series

for col in categorical_features:
    credit_card_balance_agg[f'{col}_basic'] = basic_agg_categorical_series

recency_coefficients = [0.001, 0.005]

for coeff in recency_coefficients:

    temp_df = credit_card_balance.copy()
    temp_df['RECENCY_WEIGHT'] = np.exp(coeff * temp_df['MONTHS_BALANCE'])

    weight_sums = temp_df.groupby('SK_ID_CURR')['RECENCY_WEIGHT'].sum()


    for col in numerical_features:
        weighted_feature_sum = (temp_df[col].fillna(0) * temp_df['RECENCY_WEIGHT']).groupby(temp_df['SK_ID_CURR']).sum()
        weighted_avg = np.where(weight_sums == 0, np.nan, weighted_feature_sum / weight_sums)

        col_name_suffix = str(coeff).replace(".", "")
        credit_card_balance_agg[f'{col}_{col_name_suffix}'] = weighted_avg

    for col in categorical_features:
        col_name_suffix = str(coeff).replace(".", "")
        credit_card_balance_agg[f'{col}_{col_name_suffix}'] = basic_agg_categorical_series

credit_card_balance_agg = credit_card_balance_agg.reset_index()

/tmp/ipykernel_28848/2695459537.py:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  credit_card_balance_agg[f'{col}_{col_name_suffix}'] = weighted_avg
/tmp/ipykernel_28848/2695459537.py:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  credit_card_balance_agg[f'{col}_{col_name_suffix}'] = weighted_avg
/tmp/ipykernel_28848/2695459537.py:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using p

Great! Now onto testing

In [ ]:
credit_card_balance_agg.to_parquet("credit_card_balance_agg.parquet")

In [ ]:
credit_card_balance_agg = pd.read_parquet('credit_card_balance_agg.parquet')

In [ ]:
credit_card_balance_agg.rename(columns={'index': 'SK_ID_CURR'}, inplace=True)

credit_card_balance_train = credit_card_balance_agg[
    credit_card_balance_agg['SK_ID_CURR'].isin(set(y_train.index))
].copy()

credit_card_balance_test = credit_card_balance_agg[
    credit_card_balance_agg['SK_ID_CURR'].isin(set(y_test.index))
].copy()

credit_card_balance_train['TARGET'] = credit_card_balance_train['SK_ID_CURR'].map(y_train)
credit_card_balance_test['TARGET'] = credit_card_balance_test['SK_ID_CURR'].map(y_test)

In [ ]:
categorical_cols = credit_card_balance_train.select_dtypes(include=["category", "object", "bool"]).columns.tolist()
categorical_cols = [col for col in categorical_cols if col not in ('TARGET', 'SK_ID_PREV', 'SK_ID_CURR')]
continuous_cols = credit_card_balance_train.select_dtypes(include=["float", "int"]).columns.tolist()
continuous_cols = [col for col in continuous_cols if col not in ('TARGET', 'SK_ID_PREV', 'SK_ID_CURR')]

### Mutual Information

In [ ]:
importlib.reload(sf)
sf.compare_mi(credit_card_balance_train[continuous_cols + ['TARGET']], 'TARGET')

/home/ilyac/Desktop/Programming/Credit/.venv/lib/python3.10/site-packages/sklearn/metrics/cluster/_supervised.py:49: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  type_label = type_of_target(labels_true)
/home/ilyac/Desktop/Programming/Credit/support_file.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([results_df, new_result], ignore_index=True)
/home/ilyac/Desktop/Programming/Credit/.venv/lib/python3.10/site-packages/sklearn/metrics/cluster/_supervised.py:49: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  type_label = type_of_target(labels_true)
/home/ilyac/Desktop/Programming/Credit/.venv/lib/python3.10/site-packages/sklearn/metri

,Feature,Score
0,AMT_PAYMENT_CURRENT_0001,0.194324
1,AMT_PAYMENT_CURRENT_0005,0.194318
2,AMT_PAYMENT_TOTAL_CURRENT_0005,0.193309
3,AMT_PAYMENT_TOTAL_CURRENT_0001,0.193289
4,AMT_TOTAL_RECEIVABLE_0005,0.189911
5,AMT_RECIVABLE_0005,0.189911
6,AMT_TOTAL_RECEIVABLE_0001,0.189901
7,AMT_RECIVABLE_0001,0.189901
8,AMT_TOTAL_RECEIVABLE_basic,0.189699
9,AMT_RECIVABLE_basic,0.189699


Here we have a lot of unique and useful features - great!

In [ ]:
sf.compare_mi(credit_card_balance_train[categorical_cols + ['TARGET']], 'TARGET')

/home/ilyac/Desktop/Programming/Credit/support_file.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([results_df, new_result], ignore_index=True)


,Feature,Score
0,NAME_CONTRACT_STATUS_basic,0.000026
1,NAME_CONTRACT_STATUS_0001,0.000026
2,NAME_CONTRACT_STATUS_0005,0.000026


The values here are really low

### XGBoost Feature Importance

In [ ]:
sf.compare_xgb(credit_card_balance_train[continuous_cols + ['TARGET']], 'TARGET')

/home/ilyac/Desktop/Programming/Credit/support_file.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([results_df, new_result], ignore_index=True)


,Feature,Score
0,BALANCE_TO_MIN_REG_RATIO_0005,2.624267e-02
1,DRAWINGS_PER_COUNT_basic,2.585836e-02
2,PAYMENT_COVERAGE_OF_BALANCE_basic,2.320245e-02
3,AMT_DRAWINGS_POS_CURRENT_basic,2.219183e-02
4,MIN_REG_TO_LIMIT_RATIO_basic,2.111033e-02
5,PAYMENT_COVERAGE_OF_PRINCIPAL_0005,2.074229e-02
6,AMT_DRAWINGS_ATM_CURRENT_basic,2.016555e-02
7,TOTAL_PAYMENT_MIN_REG_DIFF_PCT_basic,1.997534e-02
8,PRINCIPAL_TO_TOTAL_RECEIVABLE_RATIO_0001,1.975013e-02
9,MIN_REG_TO_LIMIT_RATIO_0001,1.939265e-02


Here we have less results I would consider significant comparing the value to the results of other XGB FI tests

### SHAP

In [ ]:
sf.compare_shap(credit_card_balance_train[continuous_cols + ['TARGET']], 'TARGET')

 98%|===================| 53160/54219 [00:19<00:00]        

,Feature,Score
0,AMT_CREDIT_LIMIT_ACTUAL_0001,2.805207e-01
1,CNT_DRAWINGS_ATM_CURRENT_0001,2.250919e-01
2,AMT_CREDIT_LIMIT_ACTUAL_basic,2.208677e-01
3,HAS_ATM_DRAWINGS_0005,1.905779e-01
4,CNT_INSTALMENT_MATURE_CUM_0005,1.643959e-01
5,MIN_REG_TO_LIMIT_RATIO_0005,1.594599e-01
6,DRAWINGS_ATM_PCT_basic,1.122278e-01
7,DRAWINGS_PER_COUNT_0005,1.101800e-01
8,CNT_DRAWINGS_ATM_PCT_basic,8.045642e-02
9,TOTAL_PAYMENT_MIN_REG_DIFF_PCT_basic,7.404257e-02


Here we have some decent values

### Summary

As in the other cases, the rule for us to keep the feature is for it to be one of the top performers in at least one of the tests with only one variation taken into account, judged subjectively

In [ ]:
to_keep = [
    'SK_ID_CURR',
    'BALANCE_TO_MIN_REG_RATIO_0005',
    'DRAWINGS_PER_COUNT_basic',
    'PAYMENT_COVERAGE_OF_BALANCE_basic',
    'AMT_DRAWINGS_POS_CURRENT_basic',
    'MIN_REG_TO_LIMIT_RATIO_basic',
    'PAYMENT_COVERAGE_OF_PRINCIPAL_0005',
    'AMT_DRAWINGS_ATM_CURRENT_basic',
    'TOTAL_PAYMENT_MIN_REG_DIFF_PCT_basic',
    'PRINCIPAL_TO_TOTAL_RECEIVABLE_RATIO_0001',
    'PAYMENT_TO_TOTAL_RECEIVABLE_RATIO_basic',
    'CNT_DRAWINGS_TO_LIMIT_RATIO_0001',
    'CURRENT_PAYMENT_MIN_REG_DIFF_PCT_0001',
    'DRAWINGS_TO_LIMIT_RATIO_basic',
    'CNT_DRAWINGS_OTHER_PCT_0001',
    'IS_DPD_DEF_0001',
    'BALANCE_TO_TOTAL_RECEIVABLE_RATIO_basic',
    'RECEIVABLE_TO_LIMIT_RATIO_0005',
    'CNT_DRAWINGS_ATM_CURRENT_0001',
    'AMT_PAYMENT_CURRENT_0001',
    'AMT_PAYMENT_TOTAL_CURRENT_0005',
    'AMT_TOTAL_RECEIVABLE_0005',
    'AMT_BALANCE_0005',
    'BALANCE_TO_LIMIT_RATIO_0005',
    'AMT_RECEIVABLE_PRINCIPAL_0005',
    'AMT_INST_MIN_REGULARITY_0001',
    'AMT_DRAWINGS_CURRENT_0001',
    'PAYMENT_TO_PRINCIPAL_RECEIVABLE_RATIO_0005',
    'AMT_CREDIT_LIMIT_ACTUAL_0001',
    'HAS_ATM_DRAWINGS_0005',
    'CNT_INSTALMENT_MATURE_CUM_0005'
]

In [ ]:
credit_card_balance_train = credit_card_balance_train[to_keep + ['TARGET']]
credit_card_balance_test = credit_card_balance_test[to_keep + ['TARGET']]

In [ ]:
credit_card_balance_train.head()

,SK_ID_CURR,BALANCE_TO_MIN_REG_RATIO_0005,DRAWINGS_PER_COUNT_basic,PAYMENT_COVERAGE_OF_BALANCE_basic,AMT_DRAWINGS_POS_CURRENT_basic,MIN_REG_TO_LIMIT_RATIO_basic,PAYMENT_COVERAGE_OF_PRINCIPAL_0005,AMT_DRAWINGS_ATM_CURRENT_basic,TOTAL_PAYMENT_MIN_REG_DIFF_PCT_basic,PRINCIPAL_TO_TOTAL_RECEIVABLE_RATIO_0001,PAYMENT_TO_TOTAL_RECEIVABLE_RATIO_basic,CNT_DRAWINGS_TO_LIMIT_RATIO_0001,CURRENT_PAYMENT_MIN_REG_DIFF_PCT_0001,DRAWINGS_TO_LIMIT_RATIO_basic,CNT_DRAWINGS_OTHER_PCT_0001,IS_DPD_DEF_0001,BALANCE_TO_TOTAL_RECEIVABLE_RATIO_basic,RECEIVABLE_TO_LIMIT_RATIO_0005,CNT_DRAWINGS_ATM_CURRENT_0001,AMT_PAYMENT_CURRENT_0001,AMT_PAYMENT_TOTAL_CURRENT_0005,AMT_TOTAL_RECEIVABLE_0005,AMT_BALANCE_0005,BALANCE_TO_LIMIT_RATIO_0005,AMT_RECEIVABLE_PRINCIPAL_0005,AMT_INST_MIN_REGULARITY_0001,AMT_DRAWINGS_CURRENT_0001,PAYMENT_TO_PRINCIPAL_RECEIVABLE_RATIO_0005,AMT_CREDIT_LIMIT_ACTUAL_0001,HAS_ATM_DRAWINGS_0005,CNT_INSTALMENT_MATURE_CUM_0005,TARGET
0,100006,0.000000,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,NaN,0.000000e+00,0.000000,0.000000,0.0,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,270000.000000,0.000000,0.000000,0
1,100011,5.110915,45000.000000,2.167712,0.0,0.021979,0.032808,2432.432432,0.126212,0.406071,-2.59325,2.894709e-07,0.078899,0.013514,0.0,0.000000,0.826284,0.267034,0.052105,4767.431188,4115.968714,48066.202214,48114.146646,0.267301,46261.852388,3824.946783,2344.714493,0.032808,163703.133990,0.011195,26.370191,0
2,100013,2.451553,25778.571429,19.077035,0.0,0.009235,0.036053,6350.000000,2.777481,0.153527,0.15909,1.470644e-06,0.847158,0.037798,0.0,0.010201,0.666667,0.097431,0.231626,6961.509313,5811.258625,15345.357664,15398.558339,0.097769,14625.351519,1309.737702,5756.726043,0.036053,130756.707891,0.044003,18.151623,0
3,100021,0.000000,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,NaN,0.000000e+00,0.000000,0.000000,0.0,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,675000.000000,0.000000,0.000000,0
4,100023,0.000000,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,NaN,0.000000e+00,0.000000,0.000000,0.0,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,135179.999760,0.000000,0.000000,0


In [ ]:
credit_card_balance_test.head()

,SK_ID_CURR,BALANCE_TO_MIN_REG_RATIO_0005,DRAWINGS_PER_COUNT_basic,PAYMENT_COVERAGE_OF_BALANCE_basic,AMT_DRAWINGS_POS_CURRENT_basic,MIN_REG_TO_LIMIT_RATIO_basic,PAYMENT_COVERAGE_OF_PRINCIPAL_0005,AMT_DRAWINGS_ATM_CURRENT_basic,TOTAL_PAYMENT_MIN_REG_DIFF_PCT_basic,PRINCIPAL_TO_TOTAL_RECEIVABLE_RATIO_0001,PAYMENT_TO_TOTAL_RECEIVABLE_RATIO_basic,CNT_DRAWINGS_TO_LIMIT_RATIO_0001,CURRENT_PAYMENT_MIN_REG_DIFF_PCT_0001,DRAWINGS_TO_LIMIT_RATIO_basic,CNT_DRAWINGS_OTHER_PCT_0001,IS_DPD_DEF_0001,BALANCE_TO_TOTAL_RECEIVABLE_RATIO_basic,RECEIVABLE_TO_LIMIT_RATIO_0005,CNT_DRAWINGS_ATM_CURRENT_0001,AMT_PAYMENT_CURRENT_0001,AMT_PAYMENT_TOTAL_CURRENT_0005,AMT_TOTAL_RECEIVABLE_0005,AMT_BALANCE_0005,BALANCE_TO_LIMIT_RATIO_0005,AMT_RECEIVABLE_PRINCIPAL_0005,AMT_INST_MIN_REGULARITY_0001,AMT_DRAWINGS_CURRENT_0001,PAYMENT_TO_PRINCIPAL_RECEIVABLE_RATIO_0005,AMT_CREDIT_LIMIT_ACTUAL_0001,HAS_ATM_DRAWINGS_0005,CNT_INSTALMENT_MATURE_CUM_0005,TARGET
28,100114,0.000000,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.0,0.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,122181.396409,0.000000,0.000000,0
34,100137,14.769534,41250.000000,0.043238,0.000000,0.037709,0.045176,42187.500000,0.128950,0.971344,0.043560,0.000003,0.096877,0.125000,0.0,0.0,1.012544,0.948159,0.996877,14358.225220,14415.941521,320003.513691,323726.807984,0.959191,310393.390161,12738.549400,42054.011832,0.045176,337500.000000,0.246263,2.649073,0
35,100138,0.000000,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.0,0.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,102972.876313,0.000000,0.000000,0
47,100166,0.000000,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.0,0.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,225000.000000,0.000000,0.000000,0
59,100197,19.385460,10053.153704,0.046899,12617.631429,0.036249,0.048833,17860.714286,-0.078703,0.961312,0.046989,0.000013,0.275430,0.090648,0.0,0.0,1.002064,0.852013,0.521154,19903.144201,14719.430183,325700.173628,326198.270689,0.853563,312918.005566,14162.198947,30394.748574,0.048833,390419.506576,0.238658,9.228575,0


In [ ]:
credit_card_balance_train.to_parquet('CREDIT_CARD_BALANCE_TRAIN.parquet')
credit_card_balance_test.to_parquet('CREDIT_CARD_BALANCE_TEST.parquet')

## Point-of-Sale Balance

In [ ]:
POS_CASH_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


Looking at this dataframe, I came to a conclusion that all of the columns are mentioned in either `installments` or `previous_application`, which is why this dataframe has no utility and has been completely ignored

## Previous Application

In [ ]:
previous_application.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
previous_application['NAME_CONTRACT_TYPE'] = previous_application['NAME_CONTRACT_TYPE'].astype('category')

previous_application['FLAG_LAST_APPL_PER_CONTRACT'] = (
    previous_application['FLAG_LAST_APPL_PER_CONTRACT'] == 'Y'
)

previous_application['NAME_CASH_LOAN_PURPOSE'] = (
    previous_application['NAME_CASH_LOAN_PURPOSE']
    .replace({'XAP': np.nan, 'XNA': np.nan})
    .astype('category')
)

previous_application['NAME_CONTRACT_STATUS'] = previous_application['NAME_CONTRACT_STATUS'].astype('category')

previous_application['NAME_PAYMENT_TYPE'] = (
    previous_application['NAME_PAYMENT_TYPE']
    .replace({'XNA': np.nan})
    .astype('category')
)

previous_application['NAME_TYPE_SUITE'] = previous_application['NAME_TYPE_SUITE'].astype('category')

previous_application['NAME_CLIENT_TYPE'] = (
    previous_application['NAME_CLIENT_TYPE']
    .replace({'XNA': np.nan})
    .astype('category')
)

previous_application['NAME_GOODS_CATEGORY'] = (
    previous_application['NAME_GOODS_CATEGORY']
    .replace({'XNA': np.nan})
    .astype('category')
)

previous_application['NAME_PORTFOLIO'] = (
    previous_application['NAME_PORTFOLIO']
    .replace({'XNA': np.nan})
    .astype('category')
)

previous_application['NAME_PRODUCT_TYPE'] = (
    previous_application['NAME_PRODUCT_TYPE']
    .replace({'XNA': np.nan})
    .astype('category')
)

previous_application['CHANNEL_TYPE'] = previous_application['CHANNEL_TYPE'].astype('category')

previous_application['NAME_SELLER_INDUSTRY'] = (
    previous_application['NAME_SELLER_INDUSTRY']
    .replace({'XNA': np.nan})
    .astype('category')
)

previous_application['NAME_YIELD_GROUP'] = (
    previous_application['NAME_YIELD_GROUP']
    .replace({'XNA': np.nan})
    .astype('category')
)

previous_application['WEEKDAY_APPR_PROCESS_START'] = previous_application['WEEKDAY_APPR_PROCESS_START'].astype('category')

previous_application['CODE_REJECT_REASON'] = (
    previous_application['CODE_REJECT_REASON']
    .replace({'XAP': np.nan, 'XNA': np.nan})
    .astype('category')
)

previous_application['PRODUCT_COMBINATION'] = previous_application['PRODUCT_COMBINATION'].astype('category')


previous_application.loc[previous_application['DAYS_FIRST_DUE'] > 0, 'DAYS_FIRST_DUE'] = np.nan
previous_application.loc[previous_application['DAYS_LAST_DUE'] > 0, 'DAYS_LAST_DUE'] = np.nan
previous_application['NFLAG_INSURED_ON_APPROVAL'] = previous_application['NFLAG_INSURED_ON_APPROVAL'].astype(bool)
previous_application['SK_ID_PREV'] = previous_application['SK_ID_PREV'].astype('int32')
previous_application['AMT_ANNUITY'] = previous_application['AMT_ANNUITY'].astype('float32')
previous_application['AMT_APPLICATION'] = previous_application['AMT_APPLICATION'].astype('float32')
previous_application['AMT_CREDIT'] = previous_application['AMT_CREDIT'].astype('float32')
previous_application['AMT_GOODS_PRICE'] = previous_application['AMT_GOODS_PRICE'].astype('float32')
previous_application['RATE_DOWN_PAYMENT'] = previous_application['RATE_DOWN_PAYMENT'].astype('float32')
previous_application['RATE_INTEREST_PRIMARY'] = previous_application['RATE_INTEREST_PRIMARY'].astype('float32')
previous_application['RATE_INTEREST_PRIVILEGED'] = previous_application['RATE_INTEREST_PRIVILEGED'].astype('float32')
previous_application['DAYS_DECISION'] = previous_application['DAYS_DECISION'].astype('int16')
previous_application['DAYS_FIRST_DUE'] = previous_application['DAYS_FIRST_DUE'].astype('float16')
previous_application['DAYS_LAST_DUE'] = previous_application['DAYS_LAST_DUE'].astype('float16')

In [ ]:
previous_application.dtypes

SK_ID_PREV                        int32
SK_ID_CURR                        int64
NAME_CONTRACT_TYPE             category
AMT_ANNUITY                     float32
AMT_APPLICATION                 float32
AMT_CREDIT                      float32
AMT_DOWN_PAYMENT                float64
AMT_GOODS_PRICE                 float32
WEEKDAY_APPR_PROCESS_START     category
HOUR_APPR_PROCESS_START           int64
FLAG_LAST_APPL_PER_CONTRACT        bool
NFLAG_LAST_APPL_IN_DAY            int64
RATE_DOWN_PAYMENT               float32
RATE_INTEREST_PRIMARY           float32
RATE_INTEREST_PRIVILEGED        float32
NAME_CASH_LOAN_PURPOSE         category
NAME_CONTRACT_STATUS           category
DAYS_DECISION                     int16
NAME_PAYMENT_TYPE              category
CODE_REJECT_REASON             category
NAME_TYPE_SUITE                category
NAME_CLIENT_TYPE               category
NAME_GOODS_CATEGORY            category
NAME_PORTFOLIO                 category
NAME_PRODUCT_TYPE              category


Now that we have fixed the datatypes, we can proceed to feature engineering

In [ ]:
previous_application['AMT_APPLICATION_TO_CREDIT_RATIO'] = previous_application['AMT_APPLICATION'] / previous_application['AMT_CREDIT']
previous_application['AMT_APPLICATION_TO_CREDIT_RATIO'].replace([np.inf, -np.inf], np.nan, inplace=True)
previous_application['AMT_APPLICATION_TO_CREDIT_RATIO'].fillna(0, inplace=True)

previous_application['AMT_CREDIT_TO_GOODS_PRICE_RATIO'] = previous_application['AMT_CREDIT'] / previous_application['AMT_GOODS_PRICE']
previous_application['AMT_CREDIT_TO_GOODS_PRICE_RATIO'].replace([np.inf, -np.inf], np.nan, inplace=True)
previous_application['AMT_CREDIT_TO_GOODS_PRICE_RATIO'].fillna(0, inplace=True)

previous_application['ANNUITY_TO_CREDIT_RATIO'] = previous_application['AMT_ANNUITY'] / previous_application['AMT_CREDIT']
previous_application['ANNUITY_TO_CREDIT_RATIO'].replace([np.inf, -np.inf], np.nan, inplace=True)
previous_application['ANNUITY_TO_CREDIT_RATIO'].fillna(0, inplace=True)

previous_application['DOWN_PAYMENT_TO_GOODS_PRICE_RATIO'] = previous_application['AMT_DOWN_PAYMENT'] / previous_application['AMT_GOODS_PRICE']
previous_application['DOWN_PAYMENT_TO_GOODS_PRICE_RATIO'].replace([np.inf, -np.inf], np.nan, inplace=True)
previous_application['DOWN_PAYMENT_TO_GOODS_PRICE_RATIO'].fillna(0, inplace=True)

previous_application['AMT_APPLICATION_TO_GOODS_PRICE_RATIO'] = previous_application['AMT_APPLICATION'] / previous_application['AMT_GOODS_PRICE']
previous_application['AMT_APPLICATION_TO_GOODS_PRICE_RATIO'].replace([np.inf, -np.inf], np.nan, inplace=True)
previous_application['AMT_APPLICATION_TO_GOODS_PRICE_RATIO'].fillna(0, inplace=True)

previous_application['LOAN_TERM_PREVIOUS'] = previous_application['DAYS_LAST_DUE'] - previous_application['DAYS_FIRST_DUE']
previous_application['LOAN_TERM_PREVIOUS'] = previous_application['LOAN_TERM_PREVIOUS'].apply(lambda x: max(0, x) if pd.notna(x) else np.nan)

previous_application['DAYS_ELAPSED_FROM_LAST_DUE_TO_TERMINATION'] = previous_application['DAYS_TERMINATION'] - previous_application['DAYS_LAST_DUE']
previous_application['DAYS_ELAPSED_FROM_LAST_DUE_TO_TERMINATION'] = previous_application['DAYS_ELAPSED_FROM_LAST_DUE_TO_TERMINATION'].apply(lambda x: max(0, x) if pd.notna(x) else np.nan)


previous_application['CONTRACT_STATUS_REJECT_REASON_COMB'] = (
    previous_application['NAME_CONTRACT_STATUS'].astype(str) + '_' +
    previous_application['CODE_REJECT_REASON'].astype(str)
)
previous_application['CONTRACT_STATUS_REJECT_REASON_COMB'] = previous_application['CONTRACT_STATUS_REJECT_REASON_COMB'].astype('category')
previous_application['PORTFOLIO_PRODUCT_TYPE_COMB'] = (
    previous_application['NAME_PORTFOLIO'].astype(str) + '_' +
    previous_application['NAME_PRODUCT_TYPE'].astype(str)
)
previous_application['PORTFOLIO_PRODUCT_TYPE_COMB'] = previous_application['PORTFOLIO_PRODUCT_TYPE_COMB'].astype('category')

previous_application['CLIENT_TYPE_CONTRACT_TYPE_COMB'] = (
    previous_application['NAME_CLIENT_TYPE'].astype(str) + '_' +
    previous_application['NAME_CONTRACT_TYPE'].astype(str)
)
previous_application['CLIENT_TYPE_CONTRACT_TYPE_COMB'] = previous_application['CLIENT_TYPE_CONTRACT_TYPE_COMB'].astype('category')

previous_application['SELLER_CHANNEL_COMB'] = (
    previous_application['NAME_SELLER_INDUSTRY'].astype(str) + '_' +
    previous_application['CHANNEL_TYPE'].astype(str)
)
previous_application['SELLER_CHANNEL_COMB'] = previous_application['SELLER_CHANNEL_COMB'].astype('category')

categorical_cols_to_encode = [
    'NAME_CONTRACT_STATUS',
    'CODE_REJECT_REASON',
    'NAME_CLIENT_TYPE',
    'NAME_PORTFOLIO'
]

categorical_cols_to_encode = [col for col in categorical_cols_to_encode if col in previous_application.columns]

if categorical_cols_to_encode:
    previous_application = pd.get_dummies(previous_application, columns=categorical_cols_to_encode, dummy_na=False)

previous_application['WAS_REJECTED'] = previous_application['NAME_CONTRACT_STATUS_Refused'].fillna(0).astype(bool) | \
                                       previous_application['NAME_CONTRACT_STATUS_Canceled'].fillna(0).astype(bool)
previous_application['WAS_REJECTED'] = previous_application['WAS_REJECTED'].astype(int)

previous_application['HAS_DOWN_PAYMENT'] = (previous_application['AMT_DOWN_PAYMENT'] > 0).astype(int)

previous_application['HAD_INSURANCE_ON_APPROVAL'] = previous_application['NFLAG_INSURED_ON_APPROVAL'].astype(int)

if 'NAME_CONTRACT_TYPE_Cash loan' in previous_application.columns:
    previous_application['HAD_PREVIOUS_CASH_LOAN'] = previous_application['NAME_CONTRACT_TYPE_Cash loan'].astype(int)
else:
    previous_application['HAD_PREVIOUS_CASH_LOAN'] = (previous_application['NAME_CONTRACT_TYPE'] == 'Cash loan').astype(int)

if 'NAME_CONTRACT_TYPE_Consumer loan' in previous_application.columns:
    previous_application['HAD_PREVIOUS_POS_LOAN'] = previous_application['NAME_CONTRACT_TYPE_Consumer loan'].astype(int)
else:
    previous_application['HAD_PREVIOUS_POS_LOAN'] = (previous_application['NAME_CONTRACT_TYPE'].str.contains('POS|Consumer loan', na=False)).astype(int)

previous_application['RATE_DOWN_PAYMENT_TO_AMT_CREDIT_RATIO'] = \
    previous_application['RATE_DOWN_PAYMENT'] / previous_application['AMT_CREDIT']

previous_application['RATE_DOWN_PAYMENT_TO_AMT_CREDIT_RATIO'].replace([np.inf, -np.inf], np.nan, inplace=True)
previous_application['RATE_DOWN_PAYMENT_TO_AMT_CREDIT_RATIO'].fillna(0, inplace=True)

previous_application['RATE_INTEREST_PRIMARY_TO_PRIVILEGED_RATIO'] = \
    previous_application['RATE_INTEREST_PRIMARY'] / previous_application['RATE_INTEREST_PRIVILEGED']

previous_application['RATE_INTEREST_PRIMARY_TO_PRIVILEGED_RATIO'].replace([np.inf, -np.inf], np.nan, inplace=True)
previous_application['RATE_INTEREST_PRIMARY_TO_PRIVILEGED_RATIO'].fillna(0, inplace=True)


/tmp/ipykernel_29681/2762248411.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  previous_application['AMT_APPLICATION_TO_CREDIT_RATIO'].replace([np.inf, -np.inf], np.nan, inplace=True)
/tmp/ipykernel_29681/2762248411.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp

In [ ]:
previous_application.dtypes

SK_ID_PREV                                      int32
SK_ID_CURR                                      int64
NAME_CONTRACT_TYPE                           category
AMT_ANNUITY                                   float32
AMT_APPLICATION                               float32
                                               ...   
HAD_INSURANCE_ON_APPROVAL                       int64
HAD_PREVIOUS_CASH_LOAN                          int64
HAD_PREVIOUS_POS_LOAN                           int64
RATE_DOWN_PAYMENT_TO_AMT_CREDIT_RATIO         float32
RATE_INTEREST_PRIMARY_TO_PRIVILEGED_RATIO     float32
Length: 69, dtype: object

And now the aggregation!

In [ ]:
group_by_col = 'SK_ID_CURR'

categorical_cols = previous_application.select_dtypes(include=["category", "object", "bool"]).columns.tolist()
categorical_cols = [col for col in categorical_cols if col not in ('SK_ID_PREV', 'SK_ID_CURR')]
continuous_cols = previous_application.select_dtypes(include=["float", "int"]).columns.tolist()
continuous_cols = [col for col in continuous_cols if col not in ('SK_ID_PREV', 'SK_ID_CURR')]

global_modes = {}
for col in categorical_cols:
    modes = previous_application[col].dropna().mode()
    if not modes.empty:
        global_modes[col] = modes.iloc[0]
    else:
        global_modes[col] = np.nan

def aggregate_categorical_mode(series, global_mode_value):
    series_cleaned = series.dropna()

    if series_cleaned.empty:
        return np.nan

    local_modes = series_cleaned.mode()

    if local_modes.empty:
        return np.nan
    elif len(local_modes) == 1:
        return local_modes.iloc[0]
    else:
        if pd.notna(global_mode_value) and global_mode_value in local_modes.values:
            return global_mode_value
        else:
            return local_modes.iloc[0]

aggregation_dict_final = {}

for col in continuous_cols:
    aggregation_dict_final[f'{col}_min'] = (col, 'min')
    aggregation_dict_final[f'{col}_max'] = (col, 'max')
    aggregation_dict_final[f'{col}_median'] = (col, 'median')

for col in categorical_cols:
    aggregation_dict_final[f'{col}_mode'] = (col, lambda x, gm_val=global_modes[col]: aggregate_categorical_mode(x, gm_val))

df_agg = previous_application.drop(columns=['SK_ID_PREV'], errors='ignore')

previous_application_agg = df_agg.groupby(group_by_col).agg(**aggregation_dict_final)

previous_application_agg.reset_index(inplace=True)

In [ ]:
previous_application_train = previous_application_agg[previous_application_agg['SK_ID_CURR'].isin(set(y_train.index))].copy()

previous_application_test = previous_application_agg[previous_application_agg['SK_ID_CURR'].isin(set(y_test.index))].copy()

previous_application_train['TARGET'] = previous_application_train['SK_ID_CURR'].map(y_train)
previous_application_test['TARGET'] = previous_application_test['SK_ID_CURR'].map(y_test)

We will use various methods to determine which features bear any actual importance. The features are going to be split into numeric and categorical and analyzed separately

In [ ]:
categorical_cols = previous_application_train.select_dtypes(include=["category", "object", "bool"]).columns.tolist()
categorical_cols = [col for col in categorical_cols if col not in ('TARGET', 'SK_ID_PREV', 'SK_ID_CURR')]
continuous_cols = previous_application_train.select_dtypes(include=["float", "int"]).columns.tolist()
continuous_cols = [col for col in continuous_cols if col not in ('TARGET', 'SK_ID_PREV', 'SK_ID_CURR')]

### Mutual Information

In [ ]:
sf.compare_mi(previous_application_train[continuous_cols + ['TARGET']], 'TARGET')

/home/ilyac/Desktop/Programming/Credit/.venv/lib/python3.10/site-packages/sklearn/metrics/cluster/_supervised.py:49: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  type_label = type_of_target(labels_true)
/home/ilyac/Desktop/Programming/Credit/support_file.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([results_df, new_result], ignore_index=True)
/home/ilyac/Desktop/Programming/Credit/.venv/lib/python3.10/site-packages/sklearn/metrics/cluster/_supervised.py:49: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  type_label = type_of_target(labels_true)
/home/ilyac/Desktop/Programming/Credit/.venv/lib/python3.10/site-packages/sklearn/metri

,Feature,Score
0,AMT_ANNUITY_median,2.341873e-01
1,AMT_ANNUITY_min,1.782069e-01
2,AMT_ANNUITY_max,1.708912e-01
3,ANNUITY_TO_CREDIT_RATIO_median,1.659728e-01
4,RATE_DOWN_PAYMENT_TO_AMT_CREDIT_RATIO_max,1.561322e-01
5,AMT_CREDIT_TO_GOODS_PRICE_RATIO_median,1.391051e-01
6,AMT_APPLICATION_TO_CREDIT_RATIO_median,1.371910e-01
7,AMT_CREDIT_median,1.322386e-01
8,AMT_APPLICATION_TO_CREDIT_RATIO_max,1.302074e-01
9,RATE_DOWN_PAYMENT_max,1.299978e-01


Here we see a lot of good values, with the ones on top being astronomically good as for mutual information

In [ ]:
sf.compare_mi(previous_application_train[categorical_cols + ['TARGET']], 'TARGET')

/home/ilyac/Desktop/Programming/Credit/support_file.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([results_df, new_result], ignore_index=True)


,Feature,Score
0,SELLER_CHANNEL_COMB_mode,2.010548e-04
1,NAME_CASH_LOAN_PURPOSE_mode,7.384821e-05
2,CONTRACT_STATUS_REJECT_REASON_COMB_mode,5.735299e-05
3,NAME_GOODS_CATEGORY_mode,4.890878e-05
4,PRODUCT_COMBINATION_mode,4.659401e-05
5,CLIENT_TYPE_CONTRACT_TYPE_COMB_mode,4.071649e-05
6,CODE_REJECT_REASON_VERIF_mode,3.666059e-05
7,CHANNEL_TYPE_mode,2.168844e-05
8,PORTFOLIO_PRODUCT_TYPE_COMB_mode,2.122894e-05
9,NAME_SELLER_INDUSTRY_mode,1.927087e-05


Significantly less here, might consider including the first one though

### Correlation Ratio

In [ ]:
sf.compare_corr(previous_application_train[continuous_cols + ['TARGET']], 'TARGET')

/home/ilyac/Desktop/Programming/Credit/support_file.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([results_df, new_result], ignore_index=True)


,Feature,Score
0,RATE_INTEREST_PRIMARY_min,9.903494e-05
1,RATE_INTEREST_PRIMARY_median,8.922434e-05
2,RATE_INTEREST_PRIMARY_max,8.744941e-05
3,RATE_INTEREST_PRIVILEGED_median,4.336823e-05
4,NFLAG_LAST_APPL_IN_DAY_median,4.150936e-05
5,RATE_INTEREST_PRIVILEGED_min,3.660020e-05
6,AMT_APPLICATION_TO_CREDIT_RATIO_max,3.623475e-05
7,DAYS_FIRST_DRAWING_median,3.202101e-05
8,DOWN_PAYMENT_TO_GOODS_PRICE_RATIO_max,3.183924e-05
9,RATE_DOWN_PAYMENT_max,3.081348e-05


Nothing worth talking about here I suppose

### XGBoost Feature Importance

In [ ]:
sf.compare_xgb(previous_application_train[continuous_cols + ['TARGET']], 'TARGET')

/home/ilyac/Desktop/Programming/Credit/support_file.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([results_df, new_result], ignore_index=True)


,Feature,Score
0,AMT_CREDIT_TO_GOODS_PRICE_RATIO_max,5.187286e-02
1,AMT_GOODS_PRICE_min,4.172514e-02
2,AMT_DOWN_PAYMENT_max,4.106129e-02
3,AMT_ANNUITY_min,3.881725e-02
4,DAYS_TERMINATION_min,3.567166e-02
5,AMT_ANNUITY_median,3.081913e-02
6,DAYS_ELAPSED_FROM_LAST_DUE_TO_TERMINATION_median,2.854221e-02
7,AMT_CREDIT_TO_GOODS_PRICE_RATIO_min,2.634241e-02
8,ANNUITY_TO_CREDIT_RATIO_max,2.550222e-02
9,HOUR_APPR_PROCESS_START_max,2.541193e-02


The values are actually pretty high as for XGBoost, we will make sure to use them

### Summary

The rules are the same, we are going to delete features that have been unequivocally voted as not useful, even after this much testing. First, we are going to keep only some of the continuous features. The criterion for that is to be near the top in at least one of the used techniques, which is pretty generous

In [ ]:
to_keep = ['SK_ID_CURR', 'AMT_ANNUITY_max', 'AMT_ANNUITY_median', 'AMT_ANNUITY_min', 'AMT_APPLICATION_TO_CREDIT_RATIO_max', 'AMT_APPLICATION_TO_CREDIT_RATIO_median', 'AMT_APPLICATION_median', 'AMT_CREDIT_TO_GOODS_PRICE_RATIO_max', 'AMT_CREDIT_TO_GOODS_PRICE_RATIO_median', 'AMT_CREDIT_max', 'AMT_CREDIT_median', 'AMT_DOWN_PAYMENT_max', 'AMT_GOODS_PRICE_median', 'AMT_GOODS_PRICE_min', 'ANNUITY_TO_CREDIT_RATIO_max', 'ANNUITY_TO_CREDIT_RATIO_median', 'DAYS_TERMINATION_min', 'DOWN_PAYMENT_TO_GOODS_PRICE_RATIO_max', 'RATE_DOWN_PAYMENT_TO_AMT_CREDIT_RATIO_max', 'RATE_DOWN_PAYMENT_max', 'RATE_DOWN_PAYMENT_median', 'RATE_INTEREST_PRIMARY_max', 'RATE_INTEREST_PRIMARY_median', 'RATE_INTEREST_PRIMARY_min']


In [ ]:
previous_application_train = previous_application_train[to_keep + ['TARGET']]
previous_application_test = previous_application_test[to_keep + ['TARGET']]

In [ ]:
previous_application_train.head()

,SK_ID_CURR,AMT_ANNUITY_max,AMT_ANNUITY_median,AMT_ANNUITY_min,AMT_APPLICATION_TO_CREDIT_RATIO_max,AMT_APPLICATION_TO_CREDIT_RATIO_median,AMT_APPLICATION_median,AMT_CREDIT_TO_GOODS_PRICE_RATIO_max,AMT_CREDIT_TO_GOODS_PRICE_RATIO_median,AMT_CREDIT_max,...,ANNUITY_TO_CREDIT_RATIO_median,DAYS_TERMINATION_min,DOWN_PAYMENT_TO_GOODS_PRICE_RATIO_max,RATE_DOWN_PAYMENT_TO_AMT_CREDIT_RATIO_max,RATE_DOWN_PAYMENT_max,RATE_DOWN_PAYMENT_median,RATE_INTEREST_PRIMARY_max,RATE_INTEREST_PRIMARY_median,RATE_INTEREST_PRIMARY_min,TARGET
1,100002,9251.775391,9251.775391,9251.775391,1.000000,1.000000,179055.00,1.000000,1.000000,179055.0,...,0.051670,-17.0,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0
2,100003,98356.992188,64567.664062,6737.310059,1.011109,0.968054,337500.00,1.150980,1.033000,1035882.0,...,0.099000,-1976.0,0.100059,0.000001,0.100061,0.050030,NaN,NaN,NaN,0
4,100005,4813.200195,4813.200195,4813.200195,1.111173,0.555587,22308.75,0.899950,0.449975,40153.5,...,0.059935,-460.0,0.100050,0.000003,0.108964,0.108964,NaN,NaN,NaN,0
5,100006,39954.511719,26636.759766,2482.919922,1.250017,0.943934,270000.00,1.316797,0.899922,906615.0,...,0.036064,-416.0,0.200011,0.000005,0.217830,0.163412,NaN,NaN,NaN,0
6,100007,22678.785156,14524.312500,1834.290039,1.175185,0.969043,191250.00,1.264000,1.033000,284400.0,...,0.098390,-2041.0,0.214042,0.000015,0.218890,0.159516,NaN,NaN,NaN,0


In [ ]:
previous_application_test.head()

,SK_ID_CURR,AMT_ANNUITY_max,AMT_ANNUITY_median,AMT_ANNUITY_min,AMT_APPLICATION_TO_CREDIT_RATIO_max,AMT_APPLICATION_TO_CREDIT_RATIO_median,AMT_APPLICATION_median,AMT_CREDIT_TO_GOODS_PRICE_RATIO_max,AMT_CREDIT_TO_GOODS_PRICE_RATIO_median,AMT_CREDIT_max,...,ANNUITY_TO_CREDIT_RATIO_median,DAYS_TERMINATION_min,DOWN_PAYMENT_TO_GOODS_PRICE_RATIO_max,RATE_DOWN_PAYMENT_TO_AMT_CREDIT_RATIO_max,RATE_DOWN_PAYMENT_max,RATE_DOWN_PAYMENT_median,RATE_INTEREST_PRIMARY_max,RATE_INTEREST_PRIMARY_median,RATE_INTEREST_PRIMARY_min,TARGET
0,100001,3951.000000,3951.000000,3951.000000,1.044079,1.044079,24835.5,0.957782,0.957782,23787.0,...,0.166099,-1612.0,0.101468,0.000004,0.104326,0.104326,NaN,NaN,NaN,0
3,100004,5357.250000,5357.250000,5357.250000,1.207699,1.207699,24282.0,0.828021,0.828021,20106.0,...,0.266450,-714.0,0.200148,0.000011,0.212008,0.212008,NaN,NaN,NaN,0
7,100008,25309.574219,15015.060547,8019.089844,1.112625,1.000000,121455.0,1.115500,0.900000,501975.0,...,0.110000,-2334.0,0.101225,0.000003,0.110243,0.108909,NaN,NaN,NaN,1
18,100019,10518.615234,10518.615234,10518.615234,0.803723,0.803723,84136.5,1.244210,1.244210,104683.5,...,0.100480,-705.0,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0
27,100029,22891.679688,22891.679688,22891.679688,1.000000,1.000000,126000.0,1.000000,1.000000,126000.0,...,0.181680,-197.0,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0


In [ ]:
previous_application_train.to_parquet('PREVIOUS_APPLICATION_TRAIN.parquet')
previous_application_test.to_parquet('PREVIOUS_APPLICATION_TEST.parquet')